In [2]:
# %% [markdown]
# FlightRank JSONs (in tar) -> ONE Parquet (one row per flight option)
# - Reads from: data/jsons_raw.tar (expects JSON files under raw/)
# - Writes to : data/all_flights.parquet
# - Keeps ALL fields:
#   * Scalar fields fully flattened into columns
#   * Array/nested-heavy fields (legs, pricings) preserved as JSON strings + counts
# - Safe parsing (missing keys -> None), robust to empty/invalid files

import os
import io
import json
import tarfile
from typing import Any, Dict, List, Optional
from tqdm import tqdm
import polars as pl

# --------- CONFIG ---------
TAR_PATH = "data/raw.tar"
MEMBER_PREFIX = "raw/"          # inside-tar path prefix
OUTPUT_PARQUET = "data/all_flights.parquet"
PARQUET_COMPRESSION = "zstd"    # zstd | snappy | lz4 | uncompressed

# --------- HELPERS ---------
def get(d: Dict[str, Any], keys: List[str], default=None):
    """Safe nested getter for dicts: get(obj, ['a','b']) -> obj['a']['b'] or default."""
    cur = d
    try:
        for k in keys:
            if cur is None:
                return default
            cur = cur.get(k)
        return cur if cur is not None else default
    except AttributeError:
        return default

def as_json(value: Any) -> Optional[str]:
    """Serialize to compact JSON string; return None on failure."""
    try:
        return json.dumps(value, ensure_ascii=False, separators=(",", ":"))
    except Exception:
        return None

def to_int(x, default=None):
    try:
        return int(x) if x is not None else default
    except Exception:
        return default

def to_float(x, default=None):
    try:
        return float(x) if x is not None else default
    except Exception:
        return default

def to_str(x, default=None):
    try:
        return str(x) if x is not None else default
    except Exception:
        return default

def ensure_list(x) -> list:
    if x is None:
        return []
    return x if isinstance(x, list) else [x]

# --------- FLATTEN LOGIC ---------
def flatten_one_option(root: Dict[str, Any], option: Dict[str, Any], source_path: str) -> Dict[str, Any]:
    """
    Produce one row per flight option.
    - Fully flatten all scalar/object fields described in your spec
    - Keep legs[] and pricings[] as JSON string columns (retain 100% info)
    """
    # top-level ids & times
    row: Dict[str, Any] = {
        "source_file": source_path,
        "root_id": to_str(root.get("$id")),
        "ranker_id": to_str(root.get("ranker_id")),
        "request_time": to_str(root.get("request_time")),
    }

    # metadata
    row.update({
        "metadata_id": to_str(get(root, ["metadata", "$id"])),
        "metadata_searchType": to_int(get(root, ["metadata", "searchType"])),
    })

    # personalData
    row.update({
        "personalData_id": to_str(get(root, ["personalData", "$id"])),
        "profileId": to_int(get(root, ["personalData", "profileId"])),
        "sex": get(root, ["personalData", "sex"]),                         # bool or None
        "bySelf": get(root, ["personalData", "bySelf"]),
        "yearOfBirth": to_int(get(root, ["personalData", "yearOfBirth"])),
        "nationality": to_int(get(root, ["personalData", "nationality"])),
        "companyID": to_int(get(root, ["personalData", "companyID"])),
        "isVip": get(root, ["personalData", "isVip"]),
        "hasAssistant": get(root, ["personalData", "hasAssistant"]),
        "isGlobal": get(root, ["personalData", "isGlobal"]),
        "frequentFlyer": to_str(get(root, ["personalData", "frequentFlyer"])),
        "position": to_str(get(root, ["personalData", "position"])),
        "pointOfSale": to_str(get(root, ["personalData", "pointOfSale"])),
        "industryEN": to_str(get(root, ["personalData", "industryEN"])),
        "grade": to_str(get(root, ["personalData", "grade"])),
    })

    # routeData
    row.update({
        "routeData_id": to_str(get(root, ["routeData", "$id"])),
        "route_requestDate": to_str(get(root, ["routeData", "requestDate"])),
        "route_searchRoute": to_str(get(root, ["routeData", "searchRoute"])),
        "route_requestDepartureDate": to_str(get(root, ["routeData", "requestDepartureDate"])),
        "route_requestReturnDate": to_str(get(root, ["routeData", "requestReturnDate"])),
    })

    # data (container)
    row.update({
        "data_id": to_str(get(root, ["data", "$id"])),
    })

    # option-level
    row.update({
        "option_service_id": to_str(option.get("$id")),
        "option_id": to_str(option.get("id")),
        "validatingCarrier": to_str(option.get("validatingCarrier")),
        "option_category": to_int(option.get("category")),
    })

    # legs (array) -> keep JSON + counts
    legs = ensure_list(option.get("legs"))
    row["legs_count"] = len(legs)
    row["legs_json"] = as_json(legs)

    # pricings (array) -> keep JSON + counts
    pricings = ensure_list(option.get("pricings"))
    row["pricings_count"] = len(pricings)
    row["pricings_json"] = as_json(pricings)

    return row

# --------- MAIN EXTRACTION ---------
rows: List[Dict[str, Any]] = []

if not os.path.exists(TAR_PATH):
    raise FileNotFoundError(f"Tar not found: {TAR_PATH}")

with tarfile.open(TAR_PATH, "r") as tar:
    members = [m for m in tar.getmembers() if m.isfile() and m.name.startswith(MEMBER_PREFIX) and m.name.endswith(".json")]

    for m in tqdm(members, desc="Reading JSONs from tar"):
        fobj = tar.extractfile(m)
        if fobj is None:
            continue
        # read safely (support UTF-8 BOM or stray whitespace)
        data_bytes = fobj.read()
        if not data_bytes:
            continue
        try:
            text = data_bytes.decode("utf-8", errors="replace").strip()
            if not text:
                continue
            root_obj = json.loads(text)
        except json.JSONDecodeError:
            # skip corrupted files
            continue

        # Some files might be arrays of searches; normalize to list of roots
        roots = root_obj if isinstance(root_obj, list) else [root_obj]

        for root in roots:
            if not isinstance(root, dict):
                continue
            options = get(root, ["data", "$values"], []) or []
            # If no options array, still keep one row to preserve the search? -> up to you.
            # Here: if empty, we still emit ONE row with option fields = None
            if not options:
                rows.append(flatten_one_option(root, {}, m.name))
            else:
                for opt in options:
                    if isinstance(opt, dict):
                        rows.append(flatten_one_option(root, opt, m.name))

# Build Polars DataFrame
df = pl.DataFrame(rows)

# (Optional) enforce consistent dtypes for booleans (Polars may infer as Object if None-heavy)
bool_cols = ["sex", "bySelf", "isVip", "hasAssistant", "isGlobal"]
for c in bool_cols:
    if c in df.columns:
        # Cast: allow nulls
        df = df.with_columns(pl.col(c).cast(pl.Boolean, strict=False))

# Write ONE Parquet
os.makedirs(os.path.dirname(OUTPUT_PARQUET), exist_ok=True)
df.write_parquet(OUTPUT_PARQUET, compression=PARQUET_COMPRESSION)

df.shape, OUTPUT_PARQUET


Reading JSONs from tar: 100%|██████████| 25/25 [00:00<00:00, 63.14it/s]


((2032, 35), 'data/all_flights.parquet')

In [2]:
# %% [markdown]
# Tools: read JSON from tar, flatten to rows (one row per pricing), keep ALL fields.
MAX_SEGMENTS = 4
MAX_LEGS = 2
import io, json, tarfile, os
from typing import Any, Dict, List, Optional
from datetime import datetime

def _get(d: Dict[str, Any], path: List[str], default=None):
    """Safe nested get with list of keys."""
    cur = d
    try:
        for k in path:
            if cur is None:
                return default
            cur = cur.get(k)
        return cur if cur is not None else default
    except AttributeError:
        return default

def _as_json(x: Any) -> Optional[str]:
    """Serialize to compact JSON string; return None on failure."""
    try:
        import json
        return json.dumps(x, ensure_ascii=False, separators=(",", ":"))
    except Exception:
        return None

def _flatten_airport(prefix: str, side: Dict[str, Any]) -> Dict[str, Any]:
    """Flatten the Airport Structure under departureFrom/arrivalTo nodes."""
    out = {}
    ap = (side or {}).get("airport") or {}
    city = ap.get("city") or {}
    country = city.get("country") or {}
    # out[f"{prefix}airport_$id"] = ap.get("$id")
    # out[f"{prefix}airport_id"] = ap.get("id")
    out[f"{prefix}airport_code"] = ap.get("code")
    out[f"{prefix}airport_iata"] = ap.get("iata")
    out[f"{prefix}airport_icao"] = ap.get("icao")
    # out[f"{prefix}airport_city_$id"] = city.get("$id")
    # out[f"{prefix}airport_city_id"] = city.get("id")
    out[f"{prefix}airport_city_code"] = city.get("code")
    out[f"{prefix}airport_city_iata"] = city.get("iata")
    # out[f"{prefix}airport_city_country_$id"] = country.get("$id")
    # out[f"{prefix}airport_city_country_id"] = country.get("id")
    out[f"{prefix}airport_city_country_codeA2"] = country.get("codeA2")
    out[f"{prefix}airport_city_country_codeA3"] = country.get("codeA3")
    out[f"{prefix}terminal"] = side.get("terminal")
    return out

def _flatten_segment(prefix: str, seg: Dict[str, Any]) -> Dict[str, Any]:
    out = {}
    # out[f"{prefix}$id"] = seg.get("$id")
    # out[f"{prefix}id"] = seg.get("id")
    out[f"{prefix}departureAt"] = seg.get("departureAt")
    out[f"{prefix}arrivalAt"] = seg.get("arrivalAt")
    out[f"{prefix}duration"] = seg.get("duration")
    out[f"{prefix}flightNumber"] = seg.get("flightNumber")
    out[f"{prefix}stopCount"] = seg.get("stopCount")
    out[f"{prefix}transfer"] = seg.get("transfer")

    # aircraft
    aircraft = seg.get("aircraft") or {}
    # out[f"{prefix}aircraft_$id"] = aircraft.get("$id")
    out[f"{prefix}aircraft_code"] = aircraft.get("code")

    # marketingCarrier
    mkt = seg.get("marketingCarrier") or {}
    # out[f"{prefix}marketingCarrier_$id"] = mkt.get("$id")
    out[f"{prefix}marketingCarrier_code"] = mkt.get("code")
    alliance = mkt.get("alliance") or {}
    # out[f"{prefix}marketingCarrier_alliance_$id"] = alliance.get("$id")
    out[f"{prefix}marketingCarrier_alliance_code"] = alliance.get("code")

    # operatingCarrier
    opt = seg.get("operatingCarrier") or {}
    # out[f"{prefix}operatingCarrier_$id"] = opt.get("$id")
    out[f"{prefix}operatingCarrier_code"] = opt.get("code")

    # airports
    out.update(_flatten_airport(f"{prefix}departureFrom_", seg.get("departureFrom") or {}))
    out.update(_flatten_airport(f"{prefix}arrivalTo_", seg.get("arrivalTo") or {}))
    return out

def _flatten_leg(prefix: str, leg: Dict[str, Any]) -> Dict[str, Any]:
    out = {}
    # out[f"{prefix}$id"] = leg.get("$id")
    out[f"{prefix}duration"] = leg.get("duration")
    out[f"{prefix}departureAt"] = leg.get("departureAt")
    out[f"{prefix}arrivalAt"] = leg.get("arrivalAt")
    out.update(_flatten_airport(f"{prefix}departureFrom_", leg.get("departureFrom") or {}))
    out.update(_flatten_airport(f"{prefix}arrivalTo_", leg.get("arrivalTo") or {}))

    # segments -> fully widen with indices; also keep raw JSON for safety
    segs = leg.get("segments") or []
    out[f"{prefix}segments_count"] = len(segs)

    # 可選：保留超過 MAX_SEGMENTS 的剩餘段（除錯/審計用；你之後若會 drop *json 欄，這欄會被丟掉也沒關係）
    # out[f"{prefix}segments_overflow_json"] = _as_json(segs[MAX_SEGMENTS:]) if len(segs) > MAX_SEGMENTS else None

    # 固定輸出 segments0..segments{MAX_SEGMENTS-1}；不足的用空 dict 讓欄位是 None
    for si in range(MAX_SEGMENTS):
        seg = segs[si] if si < len(segs) else {}
        out.update(_flatten_segment(f"{prefix}segments{si}_", seg))
    # ✅ 預先建立 faresInfo 對應欄位的占位（避免沒貼到就沒有欄）
        base = f"{prefix}segments{si}_"
        out.setdefault(base + "ticketDesignator", None)
        out.setdefault(base + "class", None)
        out.setdefault(base + "cabinClass", None)
        out.setdefault(base + "seatsAvailable", None)
        out.setdefault(base + "baggageAllowance_quantity", None)
        out.setdefault(base + "baggageAllowance_type", None)
        out.setdefault(base + "baggageAllowance_weightMeasurementType", None)
    return out

def _flatten_option_common(root: Dict[str, Any], option: Dict[str, Any]) -> Dict[str, Any]:
    """Common columns for every pricing-row under one flight option."""
    row = {}
    # top-level
    row["root_$id"] = root.get("$id")
    row["ranker_id"] = root.get("ranker_id")
    row["request_time"] = root.get("request_time")

    # metadata
    meta = root.get("metadata") or {}
    # row["metadata_$id"] = meta.get("$id")
    row["searchType"] = meta.get("searchType")

    # personalData
    pd = root.get("personalData") or {}
    row.update({
        # "personalData_$id": pd.get("$id"),
        "profileId": pd.get("profileId"),
        "sex": pd.get("sex"),
        "bySelf": pd.get("bySelf"),
        "yearOfBirth": pd.get("yearOfBirth"),
        "nationality": pd.get("nationality"),
        "companyID": pd.get("companyID"),
        "isVip": pd.get("isVip"),
        "hasAssistant": pd.get("hasAssistant"),
        "isGlobal": pd.get("isGlobal"),
        "frequentFlyer": pd.get("frequentFlyer"),
        "position": pd.get("position"),
        "pointOfSale": pd.get("pointOfSale"),
        "industryEN": pd.get("industryEN"),
        "grade": pd.get("grade"),
    })

    # routeData
    rd = root.get("routeData") or {}
    row.update({
        # "routeData_$id": rd.get("$id"),
        "requestDate": rd.get("requestDate"),
        "searchRoute": rd.get("searchRoute"),
        "requestDepartureDate": rd.get("requestDepartureDate"),
        "requestReturnDate": rd.get("requestReturnDate"),
    })

    # data container
    # data_container = root.get("data") or {}
    # row["data_$id"] = data_container.get("$id")

    # option-level
    row.update({
        # "option_$id": option.get("$id"),
        # "option_id": option.get("id"),
        "validatingCarrier": option.get("validatingCarrier"),
        "category": option.get("category"),
    })

    # legs: full widen for **this** option (indexes start 0..)
    legs = option.get("legs") or []
    # row["option_legs_count"] = len(legs)
    # row["option_legs_json"] = _as_json(legs)  # 保底全部資訊
    row["option_legs_count"] = len(legs)  # 留個計數方便檢查
    for li in range(MAX_LEGS):
        leg = legs[li] if li < len(legs) else {}
        row.update(_flatten_leg(f"legs{li}_", leg))
    return row
from collections import Counter

KNOWN_LABELS = {
    "BestPrice",
    "BestPriceDirect",
    "BestPriceTravelPolicy",
    "BestPriceCorporateTariff",
    "Convenience",
    "MinTime",
}

def _summarize_labels(labels_list):
    """labels[] -> boolean flags + counts"""
    labs = set(labels_list or [])
    out = {}
    for name in KNOWN_LABELS:
        out[f"{name}"] = (name in labs)

    return out

def _mode_or_none(items):
    items = [x for x in (items or []) if x is not None]
    if not items:
        return None
    cnt = Counter(items)
    return cnt.most_common(1)[0][0]


def build_rows_one_json(root: Dict[str, Any], source_name: str) -> List[Dict[str, Any]]:
    """
    Build rows for one root json.
    Shape: one row per pricing (offer). Keeps ALL fields via widening + JSON columns.
    """
    rows: List[Dict[str, Any]] = []
    options = _get(root, ["data", "$values"], []) or []

    # 若沒有 options，也產一列（避免資料流失）
    if not options:
        base = _flatten_option_common(root, {})
        base["pricing_present"] = False
        # base["source_file"] = source_name
        rows.append(base)
        return rows

    def set_if_none(d: Dict[str, Any], key: str, val):
        """只在還沒填過時寫入，避免重覆 fare 覆蓋。"""
        if key not in d or d[key] is None:
            d[key] = val
    for opt in options:
        base = _flatten_option_common(root, opt)
        # 構建 segment id → (leg_idx, seg_idx) 映射（供 faresInfo 對應）
        seg_id_to_pos: Dict[str, tuple] = {}
        for li, leg in enumerate(opt.get("legs") or []):
            if li >= MAX_LEGS:  # 超過 legs1 就不展開
                continue
            for si, seg in enumerate(leg.get("segments") or []):
                sid = seg.get("id")
                if sid is not None:
                    seg_id_to_pos[str(sid)] = (li, si)

        pricings = opt.get("pricings") or []
        if not pricings:
            # 沒有 pricings 也保留一列
            row = dict(base)
            row["pricing_present"] = False
            # row["source_file"] = source_name
            rows.append(row)
            continue

        for pi, pr in enumerate(pricings):
            row = dict(base)
            row["pricing_present"] = True
            # row["source_file"] = source_name

            # pricing flat
            row.update({
                # "pricing_idx": pi,
                # "pricing_$id": pr.get("$id"),
                # "pricing_id": pr.get("id"),
                "totalPrice": pr.get("totalPrice"),
                "taxes": pr.get("taxes"),
                "timeLimit": pr.get("timeLimit"),
                "supplierId": pr.get("supplierId"),
                "ordinal": pr.get("ordinal"),
                "category": pr.get("category"),
                "currencyCode": pr.get("currencyCode"),
                "corporateTariffCode": pr.get("corporateTariffCode"),
                "isSegDiscountVariant": pr.get("isSegDiscountVariant"),
                "isSegDiscountAsExactValue": pr.get("isSegDiscountAsExactValue"),
            })

            # # arrays under pricing -> JSON 欄位完整保留
            # row["labels_json"] = _as_json(pr.get("labels"))
            # row["pricing_miniRules_json"] = _as_json(pr.get("miniRules"))
            # row["pricing_pricingInfo_json"] = _as_json(pr.get("pricingInfo"))
            # row["pricing_additionalData_json"] = _as_json(pr.get("additionalData"))

            # 同時也提供快速計數欄
            # labels[] -> flags + counts
            labels_list = pr.get("labels") or []
            row.update(_summarize_labels(labels_list))

             # === miniRules：映射到 Rule 0/1，僅三個原生欄位 ===
            # Rule 0 ← category=31 (Cancellation)
            # Rule 1 ← category=33 (Exchange)
            for mr in (pr.get("miniRules") or []):
                cat = mr.get("category")
                if cat == 31:
                    ridx = 0
                elif cat == 33:
                    ridx = 1
                else:
                    continue  # 只要 31/33
                prefix = f"miniRules{ridx}_"
                set_if_none(row, prefix + "monetaryAmount", mr.get("monetaryAmount"))
                set_if_none(row, prefix + "percentage", mr.get("percentage"))
                set_if_none(row, prefix + "statusInfos", mr.get("statusInfos"))

            # === pricingInfo：依 passengerType 展開 ===
            pinfo_list = pr.get("pricingInfo") or []
            row["pricingInfo_count"] = len(pinfo_list)
            for j, pinf in enumerate(pinfo_list):
                idx = pinf.get("passengerType")
                if idx is None:
                    idx = j  # 後備索引
                base_pi = f"pricingInfo_"
                set_if_none(row, base_pi + "passengerType", pinf.get("passengerType"))
                set_if_none(row, base_pi + "price",          pinf.get("price"))
                set_if_none(row, base_pi + "taxes",          pinf.get("taxes"))
                set_if_none(row, base_pi + "passengerCount", pinf.get("passengerCount"))
                set_if_none(row, base_pi + "selfPaid",       pinf.get("selfPaid"))
                set_if_none(row, base_pi + "isAccessTP",     pinf.get("isAccessTP"))

            # 方便 join / 唯一鍵
            # UID：官方程式把特定 id 當 UID；這裡提供 option_id 與 pricing_id 的複合鍵
            # row["UID"] = row.get("pricing_id") or f"{row.get('option_id')}::pricing::{pi}"
            # === 把 faresInfo 映射回每個 segment 的 fare 欄位 ===
            # 欄位命名：leg{li}_segments{si}_fare_*
            for pinfo in (pr.get("pricingInfo") or []):
                
                for fare in (pinfo.get("faresInfo") or []):
                    bag = fare.get("baggageAllowance") or {}
                    applied_ids = fare.get("applyToSegmentIds") or []

                    for seg_id in applied_ids:
                        pos = seg_id_to_pos.get(str(seg_id))
                        if not pos:
                            continue
                        li, si = pos
                        if si >= MAX_SEGMENTS:
                            # 超過 segments3 的不再展平（仍可在 overflow_json 找到）
                            continue
                        base_key = f"legs{li}_segments{si}"

                        set_if_none(row, f"{base_key}_ticketDesignator", fare.get("ticketDesignator"))
                        set_if_none(row, f"{base_key}_class", fare.get("class"))
                        set_if_none(row, f"{base_key}_cabinClass", fare.get("cabinClass"))
                        set_if_none(row, f"{base_key}_seatsAvailable", fare.get("seatsAvailable"))
                        # set_if_none(row, f"{base_key}_fareFamilyKey", fare.get("fareFamilyKey"))

                        # 行李資訊
                        set_if_none(row, f"{base_key}_baggageAllowance_quantity", bag.get("quantity"))
                        set_if_none(row, f"{base_key}_baggageAllowance_type", bag.get("type"))
                        set_if_none(row, f"{base_key}_baggageAllowance_weightMeasurementType", bag.get("weightMeasurementType"))

            rows.append(row)

    return rows


In [8]:
# %% [markdown]
# Read ONLY JSONs whose file stem is in train_filled[ranker_id], with tqdm and memory-friendly options.

import os, json, tarfile
from typing import Any, Dict, List, Optional, Set
import polars as pl

# tqdm for progress
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs): return x

# === Paths & options ===
TAR_PATH = "data/jsons_raw.tar"
RAW_PREFIX = "raw/"
TRAIN_FILLED_PATH = "data/test.parquet"  # 你的 train_filled 檔；若你已經有變數 train_filled 可不用讀
OUTPUT_PARQUET = "data/test_all_columns.parquet"
PARQUET_COMPRESSION = "zstd"

BATCH_SIZE = 10000
# 大資料選項（二選一）
WRITE_SINGLE_PARQUET = False      # 單一檔（需要把所有批次 concat；較吃記憶體）
WRITE_DATASET_DIR    = True     # 建議給「超大資料」：每批寫一個 part 檔
DATASET_DIR = "data/test_all_columns_dataset"  # 只有在 WRITE_DATASET_DIR=True 時會用到

# === Helpers ===

def _get_ranker_id_whitelist(train_df: Optional[pl.DataFrame] = None,
                             parquet_path: Optional[str] = None) -> Set[str]:
    """
    從 train_filled 取出 ranker_id 的集合（字串型），回傳為 set。
    優先用傳入的 DataFrame；否則從 parquet_path 讀。
    """
    if train_df is None:
        if parquet_path is None or not os.path.exists(parquet_path):
            raise FileNotFoundError("請提供 train_filled DataFrame 或有效的 TRAIN_FILLED_PATH")
        train_df = pl.read_parquet(parquet_path, columns=["ranker_id"])

    ids = (train_df
           .select(pl.col("ranker_id").cast(pl.Utf8))
           .drop_nulls()
           .unique()
           .to_series()
           .to_list())
    return set(ids)

def _iter_raw_json_members(tar: tarfile.TarFile,
                           prefix: str,
                           allowed_stems: Optional[Set[str]] = None):
    """
    只走訪 tar 中 prefix 下的 .json 檔；若提供 allowed_stems，則只回傳
    檔名（不含 .json）的部份在白名單內的成員。
    """
    for m in tar.getmembers():
        if not m.isfile():
            continue
        name = m.name
        if not name.startswith(prefix) or not name.lower().endswith(".json"):
            continue
        if allowed_stems is not None:
            stem = os.path.splitext(os.path.basename(name))[0]
            if stem not in allowed_stems:
                continue
        yield m

def _load_roots_from_member(tar: tarfile.TarFile, member: tarfile.TarInfo) -> List[Dict[str, Any]]:
    """Read member -> parse JSON -> return list of root dict(s)."""
    fobj = tar.extractfile(member)
    if fobj is None:
        return []
    text = fobj.read().decode("utf-8", errors="replace").strip()
    try:
        obj = json.loads(text)
    except json.JSONDecodeError as e:
        print(f"⚠️ JSON decode failed: {member.name}: {e}")
        return []
    if isinstance(obj, list):
        return [x for x in obj if isinstance(x, dict)]
    elif isinstance(obj, dict):
        return [obj]
    else:
        return []

def _build_df_part_safely(rows: List[Dict[str, Any]]) -> pl.DataFrame:
    """
    穩健建 DataFrame：先用全量推斷；失敗則用 schema_overrides 解決混型。
    """
    if not rows:
        return pl.DataFrame([])
    try:
        return pl.DataFrame(rows, infer_schema_length=len(rows))
    except Exception as e:
        # 回退：掃描型別、建立 overrides
        type_map: Dict[str, set] = {}
        for r in rows:
            for k, v in r.items():
                if v is None:
                    continue
                type_map.setdefault(k, set()).add(type(v))
        schema_overrides: Dict[str, pl.datatypes.DataType] = {}
        for col, types in type_map.items():
            if str in types:
                schema_overrides[col] = pl.Utf8
            elif float in types:
                schema_overrides[col] = pl.Float64
            elif int in types:
                schema_overrides[col] = pl.Int64
            elif bool in types:
                schema_overrides[col] = pl.Boolean
            else:
                schema_overrides[col] = pl.Utf8
        return pl.DataFrame(rows, schema_overrides=schema_overrides,
                            infer_schema_length=len(rows))

def _drop_json_cols(df_part: pl.DataFrame) -> pl.DataFrame:
    json_cols = [c for c in df_part.columns if "json" in c.lower()]
    return df_part.drop(json_cols) if json_cols else df_part

def _cast_booleans(df_part: pl.DataFrame) -> pl.DataFrame:
    bool_suffixes = ("isVip", "hasAssistant", "isGlobal", "bySelf", "sex",
                     "selfPaid", "isAccessTP")
    bool_cols = [c for c in df_part.columns if c.endswith(bool_suffixes)]
    if not bool_cols:
        return df_part
    return df_part.with_columns([pl.col(c).cast(pl.Boolean, strict=False) for c in bool_cols])

# === Build whitelist from train_filled ===
try:
    # 若你 Notebook 內已經有 train_filled 變數就用它；否則讀 parquet
    whitelist = _get_ranker_id_whitelist(
                                         parquet_path=TRAIN_FILLED_PATH)
except Exception as e:
    raise

print(f"✅ whitelist size (distinct ranker_id): {len(whitelist)}")

# === Main ===
dfs: List[pl.DataFrame] = []
batch_rows: List[Dict[str, Any]] = []
n_files_kept = 0
n_files_total = 0
n_rows_written = 0
batch_idx = 0

# 若寫成資料集，先準備資料夾
if WRITE_DATASET_DIR:
    os.makedirs(DATASET_DIR, exist_ok=True)

with tarfile.open(TAR_PATH, "r") as tar:
    members_all = [m for m in tar.getmembers()
                   if m.isfile() and m.name.startswith(RAW_PREFIX) and m.name.lower().endswith(".json")]
    n_files_total = len(members_all)

    # 先過濾
    members = list(_iter_raw_json_members(tar, RAW_PREFIX, allowed_stems=whitelist))
    print(f"Found {n_files_total} JSON files under {RAW_PREFIX}, "
          f"filtered to {len(members)} by ranker_id whitelist.")

    for m in tqdm(members, desc="Parsing JSON members", unit="file"):
        roots = _load_roots_from_member(tar, m)
        if not roots:
            continue
        n_files_kept += 1

        for root in roots:
            batch_rows.extend(build_rows_one_json(root, m.name))

        # flush one batch
        if n_files_kept % BATCH_SIZE == 0 and batch_rows:
            batch_idx += 1
            df_part = _build_df_part_safely(batch_rows)
            df_part = _drop_json_cols(df_part)
            df_part = _cast_booleans(df_part)

            if WRITE_DATASET_DIR:
                out_part = os.path.join(DATASET_DIR, f"part-{batch_idx:06d}.parquet")
                df_part.write_parquet(out_part, compression=PARQUET_COMPRESSION)
                n_rows_written += df_part.height
            else:
                dfs.append(df_part)
            batch_rows = []

    # flush tail
    if batch_rows:
        batch_idx += 1
        df_part = _build_df_part_safely(batch_rows)
        df_part = _drop_json_cols(df_part)
        df_part = _cast_booleans(df_part)
        if WRITE_DATASET_DIR:
            out_part = os.path.join(DATASET_DIR, f"part-{batch_idx:06d}.parquet")
            df_part.write_parquet(out_part, compression=PARQUET_COMPRESSION)
            n_rows_written += df_part.height
        else:
            dfs.append(df_part)
        batch_rows = []

# === Write out ===
if WRITE_DATASET_DIR:
    print("✅ dataset mode complete")
    print("✅ parts written:", batch_idx)
    print("✅ rows written:", n_rows_written)
    print("📂 dataset dir:", DATASET_DIR)
elif WRITE_SINGLE_PARQUET:
    df = pl.concat(dfs, how="vertical", rechunk=True) if dfs else pl.DataFrame([])
    os.makedirs(os.path.dirname(OUTPUT_PARQUET), exist_ok=True)
    df.write_parquet(OUTPUT_PARQUET, compression=PARQUET_COMPRESSION)
    print("✅ files kept:", n_files_kept, "/", n_files_total)
    print("✅ rows:", df.height)
    print("✅ cols:", df.width)
    print("✅ saved to:", OUTPUT_PARQUET)


✅ whitelist size (distinct ranker_id): 45231
Found 150770 JSON files under raw/, filtered to 45231 by ranker_id whitelist.


Parsing JSON members: 100%|██████████| 45231/45231 [10:50<00:00, 69.52file/s]  


✅ dataset mode complete
✅ parts written: 5
✅ rows written: 3220271
📂 dataset dir: data/test_all_columns_dataset


In [9]:
import polars as pl
import glob, os
from collections import defaultdict
from tqdm.auto import tqdm

INPUT_GLOB = "data/test_all_columns_dataset/*.parquet"
OUTPUT_PARQUET = "data/test_all_columns.parquet"

files = sorted(glob.glob(INPUT_GLOB))
if not files:
    raise FileNotFoundError("No parquet parts found. Check your glob path.")

# 1) 蒐集所有檔案的 schema（只讀 0 列，非常快）
schemas = [pl.read_parquet(f, n_rows=0).schema for f in files]

# 2) 決定每個欄位的「目標型別」
type_sets = defaultdict(set)
for sch in schemas:
    for name, dtype in sch.items():
        type_sets[name].add(dtype)

def _is_int(t):
    return t in (pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64)

def _is_float(t):
    return t in (pl.Float32, pl.Float64)

def choose_supertype(types:set[pl.DataType]) -> pl.DataType:
    # 有 String 就用 String（解所有 Null/String 衝突）
    if pl.Utf8 in types:
        return pl.Utf8
    # 其次 Float
    if any(_is_float(t) for t in types):
        return pl.Float64
    # 再來 Int
    if any(_is_int(t) for t in types):
        return pl.Int64
    # Boolean
    if pl.Boolean in types:
        return pl.Boolean
    # Date/Datetime/Duration 等：如果只有同類就回原型別
    if len(types) == 1:
        return next(iter(types))
    # 其它混型 -> 最安全轉成 Utf8
    return pl.Utf8

target_schema = {name: choose_supertype(dts) for name, dts in type_sets.items()}
all_cols = list(target_schema.keys())
all_cols_sorted = sorted(all_cols)

# 3) 逐檔建立 LazyFrame，補齊缺欄 + cast 成目標型別
lfs = []
for f in tqdm(files, desc="files", unit="file"):
    file_schema = pl.read_parquet(f, n_rows=0).schema
    present_cols = set(file_schema.keys())
    missing_cols = [c for c in all_cols if c not in present_cols]

    lf = pl.scan_parquet(f)

    # 先 cast 現有欄位
    casts = [
        pl.col(c).cast(target_schema[c], strict=False)
        for c in present_cols
        if c in target_schema
    ]
    lf = lf.with_columns(casts) if casts else lf

    # 對於缺欄，補 None（指定 dtype，之後 concat 就不會型別衝突）
    adds = [
        pl.lit(None, dtype=target_schema[c]).alias(c)
        for c in missing_cols
    ]
    lf = lf.with_columns(adds) if adds else lf

    # 統一欄位順序（可選但建議）
    lf = lf.select([pl.col(c) for c in all_cols_sorted])

    lfs.append(lf)

# 4) 串流合併並寫單一 parquet
lf_all = pl.concat(lfs, how="vertical")
os.makedirs(os.path.dirname(OUTPUT_PARQUET), exist_ok=True)
lf_all.sink_parquet(OUTPUT_PARQUET, compression="zstd")

print("✅ saved to:", OUTPUT_PARQUET)


files: 100%|██████████| 5/5 [00:00<00:00, 137.28file/s]


✅ saved to: data/test_all_columns.parquet


In [10]:
test_filled = pl.read_parquet('data/test.parquet')
test_df = pl.read_parquet('data/test_all_columns.parquet')


In [13]:
# test_df 是一個 Polars DataFrame
import polars as pl

n = test_df.select(pl.col("ranker_id").n_unique().alias("n_unique")).item()
print("unique ranker_id:", n)


unique ranker_id: 45231


In [14]:
# test_df 是一個 Polars DataFrame
import polars as pl

n = test_filled.select(pl.col("ranker_id").n_unique().alias("n_unique")).item()
print("unique ranker_id:", n)


unique ranker_id: 45231


In [120]:
# %% [markdown]
# Read the specific JSON in tar and write ONE parquet with ALL columns.

import polars as pl

TAR_PATH = "data/raw.tar"
MEMBER_NAME = "raw/000a8ad3b98d4b41bcddcb944357e3f0.json"  # 你的測試檔
OUTPUT_PARQUET = "data/all_columns_sample.parquet"
PARQUET_COMPRESSION = "zstd"

# --- read the single JSON member ---
with tarfile.open(TAR_PATH, "r") as tar:
    m = tar.getmember(MEMBER_NAME)
    fobj = tar.extractfile(m)
    if fobj is None:
        raise FileNotFoundError(f"Cannot extract member: {MEMBER_NAME}")
    data_bytes = fobj.read()
    text = data_bytes.decode("utf-8", errors="replace").strip()
    root_obj = json.loads(text)

# normalize: file may contain a list of searches (rare)
roots = root_obj if isinstance(root_obj, list) else [root_obj]

# build rows
all_rows: List[Dict[str, Any]] = []
for root in roots:
    if isinstance(root, dict):
        all_rows.extend(build_rows_one_json(root, MEMBER_NAME))

# to Polars and write parquet
df = pl.DataFrame(all_rows)

# 可選：把布林欄位統一成 Boolean（避免全 None 推斷成 Object）
bool_cols = [c for c in df.columns if c.endswith(("isVip", "hasAssistant", "isGlobal", "bySelf", "sex"))]
if bool_cols:
    df = df.with_columns([pl.col(c).cast(pl.Boolean, strict=False) for c in bool_cols])

# 寫檔
os.makedirs(os.path.dirname(OUTPUT_PARQUET), exist_ok=True)
df.write_parquet(OUTPUT_PARQUET, compression=PARQUET_COMPRESSION)

print("✅ rows:", df.height)
print("✅ cols:", df.width)
print("✅ saved to:", OUTPUT_PARQUET)

# # 順手看一下主要欄位
# df.select([
#     "ranker_id", "routeData_searchRoute", "option_id",
#     "pricing_id", "pricing_totalPrice", "option_legs_count",
#     "pricing_labels_count", "pricing_pricingInfo_count"
# ]).head().to_pandas()


✅ rows: 88
✅ cols: 358
✅ saved to: data/all_columns_sample.parquet


In [2]:
# ========= 2) 簽名正規化：時間砍到分鐘、價錢四捨五入整數，其它轉字串，缺值→"∅" =========
def normalized_sig_exprs(cols):
    exprs = []
    for c in cols:
        e = pl.col(c)
        if c.endswith("departureAt") or c.endswith("arrivalAt"):
            # 轉字串 → 只取到分鐘（YYYY-MM-DDTHH:MM）
            e = e.cast(pl.Utf8).str.slice(0, 16)
        elif c in ("totalPrice", "taxes"):
            # 數值 → 四捨五入到整數，最後轉字串
            e = e.cast(pl.Float64).round(0).cast(pl.Int64).cast(pl.Utf8)
        else:
            e = e.cast(pl.Utf8)
        exprs.append(pl.coalesce([e, pl.lit("∅")]).alias(f"__sig__{c}"))
    return exprs

def signature_expr(cols):
    return pl.concat_str(
        [pl.col("ranker_id").cast(pl.Utf8)] + [pl.col(f"__sig__{c}") for c in cols],
        separator="|",
        ignore_nulls=False,
    ).alias("_sig")


In [ ]:
import polars as pl

TRAIN_FILLED_PATH = "data/train.parquet"

# 用你選好的 sig_cols + ranker_id 建簽名（沿用你的 normalized_sig_exprs/signature_expr）
sig_cols = [
    "totalPrice", 'taxes',
    "legs0_departureAt", "legs0_arrivalAt",
    "legs0_segments0_flightNumber",
    "legs0_segments0_marketingCarrier_code",
    "legs0_segments0_departureFrom_airport_iata",
    "legs0_segments0_arrivalTo_airport_iata",
    # 若是來回多半也能用到（存在就會被選用）
    "legs1_departureAt", "legs1_arrivalAt",
    "legs1_segments0_flightNumber",
    "legs1_segments0_marketingCarrier_code",
    "legs1_segments0_departureFrom_airport_iata",
    "legs1_segments0_arrivalTo_airport_iata",
    'legs0_segments1_flightNumber', 'legs1_segments1_flightNumber',
    'legs1_segments0_seatsAvailable','legs0_segments0_seatsAvailable',
    'legs1_segments1_seatsAvailable','legs0_segments1_seatsAvailable',
    'legs1_segments0_baggageAllowance_quantity', 'corporateTariffCode','legs0_segments0_baggageAllowance_quantity',
    'legs0_segments1_marketingCarrier_code', 'legs0_segments2_flightNumber',
    'miniRules0_monetaryAmount','miniRules1_monetaryAmount',
    'legs1_segments1_baggageAllowance_quantity', 'legs0_segments1_baggageAllowance_quantity',
    'legs1_segments2_baggageAllowance_quantity', 'legs0_segments2_baggageAllowance_quantity',
    'legs1_segments2_flightNumber', 'legs1_segments0_baggageAllowance_weightMeasurementType', 'miniRules0_statusInfos',
    'legs0_segments2_seatsAvailable',
    'legs0_segments1_duration',
]

# train_filled = pl.read_parquet(TRAIN_FILLED_PATH)

tf_sig = (
    train_filled.lazy()
    .select(["ranker_id"] + sig_cols + ["Id"])
    .with_columns(normalized_sig_exprs(sig_cols))
    .with_columns(signature_expr(sig_cols))
    .select(["_sig", "Id"])
    .collect()
)

n_rows   = tf_sig.height
n_unique = tf_sig.select(pl.col("_sig").n_unique()).item()
print("rows:", n_rows, "unique _sig:", n_unique, "collisions(rows-unique):", n_rows - n_unique)


rows: 18145372 unique _sig: 18145357 collisions(rows-unique): 15


In [15]:
tf_sig['Id']==8740001

Id
bool
false
false
false
false
false
…
false
false
false


In [14]:
dup_sig[0]['_sig']

_sig
str
"""b3367fb8be244d7bbdad008bf4755e…"


In [18]:
dup_sig = (
    tf_sig.lazy()
    .group_by("_sig")
    .agg([
        pl.col("Id").n_unique().alias("n_id"),
        pl.len().alias("rows"),
        pl.col("Id").unique().alias("ids")
    ])
    .filter(pl.col("n_id") > 1)   # 這些簽名對到多個 Id -> 真的有碰撞
    .sort("rows", descending=True)
    .collect()
)

print("有碰撞的簽名數:", dup_sig.height)
dup_sig.head(10)


有碰撞的簽名數: 15


_sig,n_id,rows,ids
str,u32,u32,list[i64]
"""4972d58d23004266855376d961974f…",2,2,"[8813520, 8813521]"
"""f02058ba6845487884ace28e7f574f…",2,2,"[8800562, 8800563]"
"""a353d0ea4848479d838eddbd04b875…",2,2,"[8782316, 8782317]"
"""cb065a680c1746328cb25d58bb12d0…",2,2,"[16575718, 16575719]"
"""3cb3bb789f744c7893c93c4cae9b48…",2,2,"[2933252, 2933253]"
"""692b8a31428f4c839a57a0d103791f…",2,2,"[3125172, 3125173]"
"""03c6e34b47dc4098bd31dd3e7c6934…",2,2,"[16575407, 16575408]"
"""0a9885227e24402c875f7c97463dcd…",2,2,"[652106, 652107]"
"""64e09d40c7c64c599842f7fb2354e9…",2,2,"[3727794, 3727795]"


In [21]:
dup_sig

_sig,n_id,rows,ids
str,u32,u32,list[i64]
"""4972d58d23004266855376d961974f…",2,2,"[8813520, 8813521]"
"""f02058ba6845487884ace28e7f574f…",2,2,"[8800562, 8800563]"
"""a353d0ea4848479d838eddbd04b875…",2,2,"[8782316, 8782317]"
"""cb065a680c1746328cb25d58bb12d0…",2,2,"[16575718, 16575719]"
"""3cb3bb789f744c7893c93c4cae9b48…",2,2,"[2933252, 2933253]"
…,…,…,…
"""84c82e4302fe4701b59bad144411ef…",2,2,"[3686817, 3686818]"
"""a056f27a7ae14e72a1e94f7caed270…",2,2,"[8360516, 8360517]"
"""130c6b1a24d94403bc96e9cf9d9841…",2,2,"[5230565, 5230566]"


In [19]:
import polars as pl
from typing import Iterable, List, Dict, Tuple, Optional

def _rank_candidates_by_separability(
    df: pl.DataFrame,
    candidates: List[str],
) -> List[Tuple[str, int, int]]:
    """
    對候選欄位評分：n_unique 越接近 N 越好、null 越少越好。
    回傳 [(col, n_unique, nulls), ...] 由好到壞排序。
    """
    N = df.height
    metas = []
    for c in candidates:
        nu = df.select(pl.col(c).n_unique()).item()
        if nu <= 1:  # 沒資訊量就跳過
            continue
        nc = df.select(pl.col(c).null_count()).item()
        metas.append((c, int(nu), int(nc)))
    # 排序：n_unique desc, null_count asc, 次要以欄名穩定排序
    metas.sort(key=lambda x: (-x[1], x[2], x[0]))
    return metas

def _unique_rows(df: pl.DataFrame, cols: List[str]) -> int:
    """計算在這些欄位上的唯一組合數。"""
    if not cols:
        return 1 if df.height > 0 else 0
    return df.select(cols).unique().height

def find_minimal_distinguishing_columns(
    view_by_ids: pl.DataFrame,
    ignore_cols: Optional[Iterable[str]] = None,
    prefer_cols_first: Optional[Iterable[str]] = None,
    max_candidates: int = 5000,
) -> Dict[str, object]:
    """
    從 view_by_ids（多列、同一 _sig 的衝突樣本）中，找出能把每列唯一區分的最小欄位組。
    - ignore_cols: 要排除的欄位（如 "Id","ranker_id","selected","_sig" ...）
    - prefer_cols_first: 想優先考慮的欄位（例如 pricing 層的 key）
    回傳：
      {
        "selected": [欄位...],               # 擬議最小分隔組
        "achieved_unique": bool,             # 是否成功把每列唯一分開
        "n_rows": int,                       # 樣本列數
        "n_unique_selected": int,            # 選定欄位組合的唯一列數
        "candidate_ranking": pl.DataFrame,   # 候選欄位的 n_unique / nulls 指標
        "preview": pl.DataFrame              # 只含 Id + 選定欄位，便於人工核對
      }
    """
    if view_by_ids.is_empty():
        return {
            "selected": [],
            "achieved_unique": True,
            "n_rows": 0,
            "n_unique_selected": 0,
            "candidate_ranking": pl.DataFrame([]),
            "preview": pl.DataFrame([]),
        }

    N = view_by_ids.height
    ignore = set(ignore_cols or []) | {"_sig"}
    # 候選欄位：剔除 ignore，且限制上限數量避免 OOM
    candidates = [c for c in view_by_ids.columns if c not in ignore]
    # 只保留在樣本中有變化的欄位
    candidates_metrics = _rank_candidates_by_separability(view_by_ids, candidates)
    candidates = [c for c, _, _ in candidates_metrics][:max_candidates]

    # 把 prefer_cols_first（若存在且在 candidates 中）提前
    if prefer_cols_first:
        pref = [c for c in prefer_cols_first if c in candidates]
        rest = [c for c in candidates if c not in pref]
        candidates = pref + rest

    # === 貪婪選擇 ===
    selected: List[str] = []
    current_unique = _unique_rows(view_by_ids, selected)

    while current_unique < N and candidates:
        best_col = None
        best_unique = current_unique
        # 從剩餘候選中挑「能讓 unique 數長最多」的那個
        # 若同分，優先現有排序（已經有把 prefer 放最前面）
        for c in candidates:
            u = _unique_rows(view_by_ids, selected + [c])
            if u > best_unique:
                best_unique = u
                best_col = c
                if u == N:  # 已達到完全分開，直接結束這輪挑選
                    break
        if best_col is None:
            # 沒有任何欄位能增加 unique 數（可能真的完全相同列）
            break
        selected.append(best_col)
        candidates.remove(best_col)
        current_unique = best_unique

    # === 簡化（剃掉冗餘） ===
    # 逐一嘗試移除，如果仍能維持唯一，就刪掉它
    pruned = True
    while pruned and len(selected) > 1:
        pruned = False
        for c in list(selected):  # 複製一份避免疊代時修改
            trial = [x for x in selected if x != c]
            if _unique_rows(view_by_ids, trial) == current_unique:
                selected = trial
                pruned = True

    # candidate ranking 給你參考
    cand_rank_df = pl.DataFrame(
        [{"column": c, "n_unique": nu, "nulls": nc} for c, nu, nc in _rank_candidates_by_separability(view_by_ids, [m[0] for m in candidates_metrics])]
    ).sort(["n_unique", "nulls"], descending=[True, False])

    # 預覽表
    show_cols = (["Id"] if "Id" in view_by_ids.columns else []) + selected
    preview = view_by_ids.select(show_cols).sort(show_cols)

    return {
        "selected": selected,
        "achieved_unique": (current_unique == N),
        "n_rows": N,
        "n_unique_selected": current_unique,
        "candidate_ranking": cand_rank_df,
        "preview": preview,
    }


In [ ]:
legs1_segments2_flightNumber legs1_segments0_baggageAllowance_weightMeasurementType miniRules0_statusInfos legs0_segments2_seatsAvailable

In [20]:
import polars as pl

k = 4# 想看的第幾個碰撞簽名（0-based）

if dup_sig.height == 0:
    raise ValueError("目前沒有任何 _sig 發生碰撞（n_id>1）。")

# 取第 k 個 _sig 與對應的 Id 清單
row_k    = dup_sig.row(k, named=True)
sig_val  = row_k["_sig"]
ids_list = row_k["ids"]  # 這是該簽名對應到的多個 Id

print(f"🔎 inspect _sig[{k}]: {sig_val}")
print(f"   ids: {ids_list}")

# 方案1：直接用 Id 清單在 train_filled 裡撈完整列
view_by_ids = train_filled.filter(pl.col("Id").is_in(ids_list))
print("=== rows by ids ===")
print(view_by_ids)
# view_by_ids 是你先前挑出同一 _sig 的那些列（多個 Id）
# 建議排除這些鍵，避免把 Id 自己也拿來分列 😄
IGNORE = {"Id", "ranker_id", "selected",'__index_level_0__'}



res = find_minimal_distinguishing_columns(
    view_by_ids,
    ignore_cols=IGNORE,
    # prefer_cols_first=PREFER,
)

print("🎯 minimal columns:", res["selected"])
print(f"unique with selected: {res['n_unique_selected']} / rows: {res['n_rows']}  → achieved_unique={res['achieved_unique']}")
print("\nTop varying candidates:")
print(res["candidate_ranking"].head(20))

print("\nPreview (Id + selected columns):")
print(res["preview"])


🔎 inspect _sig[4]: 3cb3bb789f744c7893c93c4cae9b4824|42439|8314|2024-09-22T08:45|2024-09-22T13:10|1161|OM|UBN|IST|∅|∅|∅|∅|∅|∅|∅|∅|∅|4.0|∅|∅|∅|∅|1.0|∅|∅|0.0|3500.0|∅|∅|∅|∅|∅|∅|1.0|∅|∅|0.0|∅
   ids: [2933252, 2933253]
=== rows by ids ===
shape: (2, 127)
┌─────────┬────────┬───────────┬───────────────┬───┬────────┬────────────┬──────────┬──────────────┐
│ Id      ┆ bySelf ┆ companyID ┆ corporateTari ┆ … ┆ taxes  ┆ totalPrice ┆ selected ┆ __index_leve │
│ ---     ┆ ---    ┆ ---       ┆ ffCode        ┆   ┆ ---    ┆ ---        ┆ ---      ┆ l_0__        │
│ i64     ┆ bool   ┆ i64       ┆ ---           ┆   ┆ f64    ┆ f64        ┆ i64      ┆ ---          │
│         ┆        ┆           ┆ i64           ┆   ┆        ┆            ┆          ┆ i64          │
╞═════════╪════════╪═══════════╪═══════════════╪═══╪════════╪════════════╪══════════╪══════════════╡
│ 2933252 ┆ true   ┆ 44191     ┆ null          ┆ … ┆ 8314.0 ┆ 42439.0    ┆ 0        ┆ 2933252      │
│ 2933253 ┆ true   ┆ 44191     ┆ null     

ColumnNotFoundError: unable to find column "n_unique"; valid columns: []

In [ ]:
import polars as pl
from collections import Counter

# 先在 train_filled 上補回 _sig（一次算好，後面迴圈就快）
tf_full_sig = (
    train_filled.lazy()
    .with_columns(normalized_sig_exprs(sig_cols))
    .with_columns(signature_expr(sig_cols))
    .collect()
)

# 忽略的欄位（避免拿鍵/標記欄來「分列」）
IGNORE = {"Id", "ranker_id", "selected", "_sig", "__index_level_0__"}


results = []
feat_counter = Counter()

# 逐一處理每個發生碰撞的 _sig
for row in dup_sig.iter_rows(named=True):
    s = row["_sig"]
    ids_list = row["ids"]  # 這個簽名底下對到的多個 Id（供參考）

    # 取出該簽名的所有列（完整欄位）
    view_by_sig = tf_full_sig.filter(pl.col("_sig") == s)

    # 尋找能把該群完全分開的「最小欄位組」
    res = find_minimal_distinguishing_columns(
        view_by_sig,
        ignore_cols=IGNORE,
    )

    # 記錄結果
    results.append({
        "_sig": s,
        "ids": ids_list,
        "n_rows": res["n_rows"],
        "achieved_unique": res["achieved_unique"],
        "n_unique_selected": res["n_unique_selected"],
        "selected": res["selected"],     # 這就是該群要用哪些 features 才能完全分開
    })

    # 累計每個欄位被選中的次數（跨群彙總）
    for c in res["selected"]:
        feat_counter[c] += 1

# 彙總成表：每個碰撞群對應的「最小特徵組」
per_sig_minimal = pl.DataFrame(results)
print("📌 每個碰撞簽名對應的最小欄位組（前 5 筆預覽）：")
print(per_sig_minimal.head())

# 把 selected 展開成 long 格式，統計哪個欄位最常被用來分群
feat_usage = (
    per_sig_minimal
    .explode("selected")
    .group_by("selected")
    .len()
    .sort("len", descending=True)
    .rename({"len": "used_in_groups"})
)
print("\n🏆 最常被用來分開群組的欄位（前 30）：")
print(feat_usage.head(30))

# 簡單成效統計
total_groups = per_sig_minimal.height
ok_groups    = per_sig_minimal.filter(pl.col("achieved_unique")).height
print(f"\n🎯 完全分開的群組：{ok_groups}/{total_groups}（{ok_groups/max(1,total_groups):.1%}）")

# 如需把結果存檔：
# per_sig_minimal.write_parquet("data/per_sig_minimal_features.parquet", compression="zstd")
# feat_usage.write_parquet("data/feature_usage_counts.parquet", compression="zstd")


ColumnNotFoundError: unable to find column "n_unique"; valid columns: []

In [9]:
view_by_ids

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64
6003062,true,60545,null,"""LH""",21,false,false,"""2024-08-07T00:40:00""","""2024-08-05T20:05:00""","""1.03:35:00""","""320""","""IST""","""IST""",30.0,1.0,2.0,"""LJU""","""02:25:00""","""1064""","""TK""","""TK""",3.0,"""32Q""","""AYT""","""AYT""",30.0,1.0,2.0,"""IST""","""01:35:00""","""2408""","""TK""","""TK""",4.0,"""32B""","""MOW""",…,1.0,2.0,"""IST""","""02:15:00""","""1063""","""TK""","""TK""",3.0,null,null,null,null,null,null,null,null,null,null,null,null,9350.0,null,1.0,18700.0,null,1.0,1.0,1,3001967,"""7726d08a053040f48c58002f3f6763…",2024-07-29 17:28:44,"""LJUMOW/MOWLJU""",true,55615.0,181560.0,0,6003062
6003063,true,60545,null,"""LH""",21,false,false,"""2024-08-07T00:40:00""","""2024-08-05T20:05:00""","""1.03:35:00""","""320""","""IST""","""IST""",30.0,1.0,2.0,"""LJU""","""02:25:00""","""1064""","""TK""","""TK""",3.0,"""32Q""","""AYT""","""AYT""",30.0,1.0,2.0,"""IST""","""01:35:00""","""2408""","""TK""","""TK""",4.0,"""32B""","""MOW""",…,1.0,2.0,"""IST""","""02:15:00""","""1063""","""TK""","""TK""",4.0,null,null,null,null,null,null,null,null,null,null,null,null,9350.0,null,1.0,18700.0,null,1.0,1.0,1,3001967,"""7726d08a053040f48c58002f3f6763…",2024-07-29 17:28:44,"""LJUMOW/MOWLJU""",true,55615.0,181560.0,0,6003063
6003065,true,60545,null,"""LH""",21,false,false,"""2024-08-07T00:40:00""","""2024-08-05T20:05:00""","""1.03:35:00""","""320""","""IST""","""IST""",30.0,1.0,2.0,"""LJU""","""02:25:00""","""1064""","""TK""","""TK""",3.0,"""32Q""","""AYT""","""AYT""",30.0,1.0,2.0,"""IST""","""01:35:00""","""2408""","""TK""","""TK""",4.0,"""32B""","""MOW""",…,1.0

In [ ]:
if dup_sig.height:
    bad_sig = dup_sig.select("_sig").to_series()[0]     # 取第一個問題簽名
    probe = (
        train_filled.lazy()
        .select(["ranker_id"] + sig_cols + ["Id", "supplierId","ordinal","timeLimit","currencyCode"])
        .with_columns(normalized_sig_exprs(sig_cols))
        .with_columns(signature_expr(sig_cols))
        .filter(pl.col("_sig") == bad_sig)
        .collect()
    )
    probe


In [2]:
sig_cols

['totalPrice',
 'taxes',
 'legs0_departureAt',
 'legs0_arrivalAt',
 'legs0_segments0_flightNumber',
 'legs0_segments0_marketingCarrier_code',
 'legs0_segments0_departureFrom_airport_iata',
 'legs0_segments0_arrivalTo_airport_iata',
 'legs1_departureAt',
 'legs1_arrivalAt',
 'legs1_segments0_flightNumber',
 'legs1_segments0_marketingCarrier_code',
 'legs1_segments0_departureFrom_airport_iata',
 'legs1_segments0_arrivalTo_airport_iata',
 'legs0_segments1_flightNumber',
 'legs1_segments1_flightNumber',
 'legs1_segments0_seatsAvailable',
 'legs0_segments0_seatsAvailable',
 'legs1_segments1_seatsAvailable',
 'legs0_segments1_seatsAvailable',
 'legs1_segments0_baggageAllowance_quantity',
 'corporateTariffCode',
 'legs0_segments0_baggageAllowance_quantity',
 'legs0_segments1_marketingCarrier_code',
 'legs0_segments2_flightNumber',
 'miniRules0_monetaryAmount',
 'miniRules1_monetaryAmount',
 'legs1_segments1_baggageAllowance_quantity',
 'legs0_segments1_baggageAllowance_quantity',
 'legs1_segm

In [4]:
import polars as pl
import os

TRAIN_FILLED_PATH = "data/test.parquet"
DF_PATH           = "data/test_all_columns.parquet"   # 很大的 df
OUT_PATH          = "data/test_all_columns_with_id.parquet"
SIDE_MAP_PATH     = "data/_id_side_map.parquet"        # 臨時側檔：(_idx, Id)

# ========= 1) 讀 train_filled（較小），決定可用的簽名欄 =========
train_filled = pl.read_parquet(TRAIN_FILLED_PATH)
df_schema = pl.read_parquet(DF_PATH, n_rows=0).schema
df_cols   = set(df_schema.keys())

# 一組「穩定的」候選欄位（盡量用第一段航段與第一個 segment；避免 duration 這類容易格式化不同的欄位）
SIGNATURE_CANDIDATES  = [
    "totalPrice", 'taxes',
    "legs0_departureAt", "legs0_arrivalAt",
    "legs0_segments0_flightNumber",
    "legs0_segments0_marketingCarrier_code",
    "legs0_segments0_departureFrom_airport_iata",
    "legs0_segments0_arrivalTo_airport_iata",
    # 若是來回多半也能用到（存在就會被選用）
    "legs1_departureAt", "legs1_arrivalAt",
    "legs1_segments0_flightNumber",
    "legs1_segments0_marketingCarrier_code",
    "legs1_segments0_departureFrom_airport_iata",
    "legs1_segments0_arrivalTo_airport_iata",
    'legs0_segments1_flightNumber', 'legs1_segments1_flightNumber',
    # 'legs1_segments0_seatsAvailable','legs0_segments0_seatsAvailable',
    # 'legs1_segments1_seatsAvailable',#'legs0_segments1_baggageAllowance_quantity','legs0_segments1_seatsAvailable' , "miniRules0_monetaryAmount", "miniRules1_monetaryAmount","miniRules0_statusInfos"
    # 'legs1_segments0_baggageAllowance_quantity', 
    'corporateTariffCode',
    # 'legs0_segments0_baggageAllowance_quantity',
    'legs0_segments1_marketingCarrier_code', 'legs0_segments2_flightNumber',
    # 'legs1_segments1_baggageAllowance_quantity', #'legs0_segments1_baggageAllowance_quantity',
    # 'legs1_segments2_baggageAllowance_quantity', #'legs0_segments2_baggageAllowance_quantity',
    'legs1_segments2_flightNumber', #'legs1_segments0_baggageAllowance_weightMeasurementType', 
    # 'legs0_segments2_seatsAvailable',
    'legs0_segments1_duration',
]

# 只取雙方都存在的
sig_cols = [c for c in SIGNATURE_CANDIDATES if c in train_filled.columns and c in df_cols]

if len(sig_cols) < 4:
    # 後備：用交集裡的前 N 個（避免欄太少）
    common_cols = [c for c in train_filled.columns if c in df_cols and c not in {"Id", "ranker_id"}]
    sig_cols = sorted(common_cols)[:8]  # 至少湊一些
print(f"🔑 signature columns: {sig_cols}")

# ========= 2) 簽名正規化（修正版）=========
def normalized_sig_exprs(cols):
    """
    - departureAt/arrivalAt：擷取到分鐘 "YYYY-MM-DDTHH:MM"
    - duration：從 "HH:MM:SS" 轉總分鐘（整數字串）
    - 其他：轉字串並去掉尾端無資訊的 ".0"
    - 缺值 → "∅"
    """
    exprs = []
    for c in cols:
        e = pl.col(c)

        if c.endswith("departureAt") or c.endswith("arrivalAt"):
            e = e.cast(pl.Utf8).str.extract(r"^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})", 1)

        elif c.endswith("duration"):
            s  = e.cast(pl.Utf8)
            hh = s.str.extract(r"^(\d+):", 1).cast(pl.Int64, strict=False)
            mm = s.str.extract(r"^\d+:(\d{2})", 1).cast(pl.Int64, strict=False)
            total_min = (hh * pl.lit(60) + mm)
            e = pl.when(hh.is_not_null() & mm.is_not_null()) \
                 .then(total_min.cast(pl.Int64).cast(pl.Utf8)) \
                 .otherwise(None)

        else:
            e = e.cast(pl.Utf8).str.replace(r"\.0+$", "")  # "9.0" -> "9"

        exprs.append(pl.coalesce([e, pl.lit("∅")]).alias(f"__sig__{c}"))
    return exprs

def signature_expr(cols):
    return pl.concat_str(
        [pl.col("ranker_id").cast(pl.Utf8)] + [pl.col(f"__sig__{c}") for c in cols],
        separator="|", ignore_nulls=False
    ).alias("_sig")


# ========= 3) (ranker_id, _sig, pos) -> Id 對照表（train_filled 端；用 rank 產生 pos）=========
train_sig = (
    train_filled.lazy()
    .with_row_index("_rid")                                  # 穩定順序
    .select(["ranker_id", "Id", "_rid"] + sig_cols)
    .with_columns(normalized_sig_exprs(sig_cols))
    .with_columns(signature_expr(sig_cols))                  # 產生 _sig
    .with_columns([
        # 在每個 (ranker_id, _sig) 內，依 _rid 做 ordinal rank，得到 1..n，再 -1 變 0..n-1
        (pl.col("_rid")
           .rank(method="ordinal")
           .over(["_sig"])
           .cast(pl.Int64) - 1
        ).alias("pos")
    ])
    .select(["ranker_id", "_sig", "pos", "Id"])
    .collect()
)

print(f"🗺️ mapping rows (with position): {train_sig.height:,}")


# ========= 4) 大表端也產生 (ranker_id, _sig, pos)，再 join =========
lf_small = pl.scan_parquet(DF_PATH).select(["ranker_id"] + sig_cols)

lf_side = (
    lf_small
    .with_row_index("_idx")                                  # 穩定順序
    .with_columns(normalized_sig_exprs(sig_cols))
    .with_columns(signature_expr(sig_cols))                  # 產生 _sig
    .with_columns([
        (pl.col("_idx")
           .rank(method="ordinal")
           .over(["_sig"])
           .cast(pl.Int64) - 1
        ).alias("pos")
    ])
    .join(train_sig.lazy(), on=["ranker_id", "_sig", "pos"], how="left")
    .select(["_idx", "Id"])
)



os.makedirs(os.path.dirname(SIDE_MAP_PATH), exist_ok=True)
lf_side.sink_parquet(SIDE_MAP_PATH, compression="zstd")
print(f"🧭 side map (multiplicity-aware) written → {SIDE_MAP_PATH}")

# （可選）看一下配對率
import polars as pl

side = pl.read_parquet(SIDE_MAP_PATH)  # 欄位：_idx, Id

n_rows   = side.height
n_null   = side["Id"].null_count()
n_nn     = n_rows - n_null
n_unique = side.filter(pl.col("Id").is_not_null()).select(pl.col("Id").n_unique()).item()
dupes    = n_nn - n_unique

# 各 Id 出現次數（只看非空）
counts = (
    side.filter(pl.col("Id").is_not_null())
        .group_by("Id")
        .len()
        .sort("len", descending=True)
)

max_count = counts.select(pl.col("len").max()).item() if counts.height else 0

print(f"rows(total): {n_rows:,}")
print(f"Id null    : {n_null:,}")
print(f"Id non-null: {n_nn:,}")
print(f"Id unique  : {n_unique:,}")
print(f"Id duplicates (non-null - unique): {dupes:,}")
print(f"max occurrences of a single Id   : {max_count:,}")

# 列出前 10 個重複 Id 的樣子
top_dupes = counts.filter(pl.col("len") > 1).head(10)
if top_dupes.height:
    sample = (
        side.join(top_dupes.select("Id"), on="Id", how="inner")
            .group_by("Id")
            .agg([
                pl.len().alias("n"),
                pl.col("_idx").head(5).alias("sample_idx")  # 每個 Id 範例的前 5 個 _idx
            ])
            .sort("n", descending=True)
    )
    print("\nTop duplicate Ids (n, sample_idx):")
    print(sample)

# 簡答你的問題：
if max_count == n_rows and n_rows > 0:
    print("\n⚠️ 注意：max_count == len(side)。代表所有列都被指到同一個 Id，這幾乎可以確定是簽名撞在一起了。")
elif max_count > 1:
    print("\nℹ️ 有些 Id 對到多於一列（簽名不夠唯一）；理想情況是 max_count == 1。")
else:
    print("\n✅ 每個 Id 只對到一列（max_count == 1），這是理想狀態。")


🔑 signature columns: ['totalPrice', 'taxes', 'legs0_departureAt', 'legs0_arrivalAt', 'legs0_segments0_flightNumber', 'legs0_segments0_marketingCarrier_code', 'legs0_segments0_departureFrom_airport_iata', 'legs0_segments0_arrivalTo_airport_iata', 'legs1_departureAt', 'legs1_arrivalAt', 'legs1_segments0_flightNumber', 'legs1_segments0_marketingCarrier_code', 'legs1_segments0_departureFrom_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs0_segments1_flightNumber', 'legs1_segments1_flightNumber', 'corporateTariffCode', 'legs0_segments1_marketingCarrier_code', 'legs0_segments2_flightNumber', 'legs1_segments2_flightNumber', 'legs0_segments1_duration']
🗺️ mapping rows (with position): 6,897,776
🧭 side map (multiplicity-aware) written → data/_id_side_map.parquet
rows(total): 3,220,271
Id null    : 25,051
Id non-null: 3,195,220
Id unique  : 3,195,220
Id duplicates (non-null - unique): 0
max occurrences of a single Id   : 1

✅ 每個 Id 只對到一列（max_count == 1），這是理想狀態。


In [3]:
import polars as pl
import os

TRAIN_FILLED_PATH = "data/train.parquet"
DF_PATH           = "data/train_all_columns.parquet"   # 很大的 df
OUT_PATH          = "data/train_all_columns_with_id.parquet"
SIDE_MAP_PATH     = "data/_id_side_map.parquet"        # 臨時側檔：(_idx, Id)

# ========= 1) 讀 train_filled（較小），決定可用的簽名欄 =========
train_filled = pl.read_parquet(TRAIN_FILLED_PATH)
df_schema = pl.read_parquet(DF_PATH, n_rows=0).schema
df_cols   = set(df_schema.keys())

# 一組「穩定的」候選欄位（盡量用第一段航段與第一個 segment；避免 duration 這類容易格式化不同的欄位）
SIGNATURE_CANDIDATES  = [
    "totalPrice", 'taxes',
    "legs0_departureAt", "legs0_arrivalAt",
    "legs0_segments0_flightNumber",
    "legs0_segments0_marketingCarrier_code",
    "legs0_segments0_departureFrom_airport_iata",
    "legs0_segments0_arrivalTo_airport_iata",
    # 若是來回多半也能用到（存在就會被選用）
    "legs1_departureAt", "legs1_arrivalAt",
    "legs1_segments0_flightNumber",
    "legs1_segments0_marketingCarrier_code",
    "legs1_segments0_departureFrom_airport_iata",
    "legs1_segments0_arrivalTo_airport_iata",
    'legs0_segments1_flightNumber', 'legs1_segments1_flightNumber',
    # 'legs1_segments0_seatsAvailable','legs0_segments0_seatsAvailable',
    # 'legs1_segments1_seatsAvailable',#'legs0_segments1_baggageAllowance_quantity','legs0_segments1_seatsAvailable' , "miniRules0_monetaryAmount", "miniRules1_monetaryAmount","miniRules0_statusInfos"
    # 'legs1_segments0_baggageAllowance_quantity', 
    'corporateTariffCode',
    # 'legs0_segments0_baggageAllowance_quantity',
    'legs0_segments1_marketingCarrier_code', 'legs0_segments2_flightNumber',
    # 'legs1_segments1_baggageAllowance_quantity', #'legs0_segments1_baggageAllowance_quantity',
    # 'legs1_segments2_baggageAllowance_quantity', #'legs0_segments2_baggageAllowance_quantity',
    'legs1_segments2_flightNumber', #'legs1_segments0_baggageAllowance_weightMeasurementType', 
    # 'legs0_segments2_seatsAvailable',
    'legs0_segments1_duration',
]

# 只取雙方都存在的
sig_cols = [c for c in SIGNATURE_CANDIDATES if c in train_filled.columns and c in df_cols]

if len(sig_cols) < 4:
    # 後備：用交集裡的前 N 個（避免欄太少）
    common_cols = [c for c in train_filled.columns if c in df_cols and c not in {"Id", "selected", "ranker_id"}]
    sig_cols = sorted(common_cols)[:8]  # 至少湊一些
print(f"🔑 signature columns: {sig_cols}")

# ========= 2) 簽名正規化（修正版）=========
def normalized_sig_exprs(cols):
    """
    - departureAt/arrivalAt：擷取到分鐘 "YYYY-MM-DDTHH:MM"
    - duration：從 "HH:MM:SS" 轉總分鐘（整數字串）
    - 其他：轉字串並去掉尾端無資訊的 ".0"
    - 缺值 → "∅"
    """
    exprs = []
    for c in cols:
        e = pl.col(c)

        if c.endswith("departureAt") or c.endswith("arrivalAt"):
            e = e.cast(pl.Utf8).str.extract(r"^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})", 1)

        elif c.endswith("duration"):
            s  = e.cast(pl.Utf8)
            hh = s.str.extract(r"^(\d+):", 1).cast(pl.Int64, strict=False)
            mm = s.str.extract(r"^\d+:(\d{2})", 1).cast(pl.Int64, strict=False)
            total_min = (hh * pl.lit(60) + mm)
            e = pl.when(hh.is_not_null() & mm.is_not_null()) \
                 .then(total_min.cast(pl.Int64).cast(pl.Utf8)) \
                 .otherwise(None)

        else:
            e = e.cast(pl.Utf8).str.replace(r"\.0+$", "")  # "9.0" -> "9"

        exprs.append(pl.coalesce([e, pl.lit("∅")]).alias(f"__sig__{c}"))
    return exprs

def signature_expr(cols):
    return pl.concat_str(
        [pl.col("ranker_id").cast(pl.Utf8)] + [pl.col(f"__sig__{c}") for c in cols],
        separator="|", ignore_nulls=False
    ).alias("_sig")


# ========= 3) (ranker_id, _sig, pos) -> Id 對照表（train_filled 端；用 rank 產生 pos）=========
train_sig = (
    train_filled.lazy()
    .with_row_index("_rid")                                  # 穩定順序
    .select(["ranker_id","selected", "Id", "_rid"] + sig_cols)
    .with_columns(normalized_sig_exprs(sig_cols))
    .with_columns(signature_expr(sig_cols))                  # 產生 _sig
    .with_columns([
        # 在每個 (ranker_id, _sig) 內，依 _rid 做 ordinal rank，得到 1..n，再 -1 變 0..n-1
        (pl.col("_rid")
           .rank(method="ordinal")
           .over(["_sig"])
           .cast(pl.Int64) - 1
        ).alias("pos")
    ])
    .select(["ranker_id", "_sig", "pos", "Id","selected"])
    .collect()
)

print(f"🗺️ mapping rows (with position): {train_sig.height:,}")


# ========= 4) 大表端也產生 (ranker_id, _sig, pos)，再 join =========
lf_small = pl.scan_parquet(DF_PATH).select(["ranker_id"] + sig_cols)

lf_side = (
    lf_small
    .with_row_index("_idx")                                  # 穩定順序
    .with_columns(normalized_sig_exprs(sig_cols))
    .with_columns(signature_expr(sig_cols))                  # 產生 _sig
    .with_columns([
        (pl.col("_idx")
           .rank(method="ordinal")
           .over(["_sig"])
           .cast(pl.Int64) - 1
        ).alias("pos")
    ])
    .join(train_sig.lazy(), on=["ranker_id", "_sig", "pos"], how="left")
    .select(["_idx", "Id", "selected"])
)



os.makedirs(os.path.dirname(SIDE_MAP_PATH), exist_ok=True)
lf_side.sink_parquet(SIDE_MAP_PATH, compression="zstd")
print(f"🧭 side map (multiplicity-aware) written → {SIDE_MAP_PATH}")

# （可選）看一下配對率
import polars as pl

side = pl.read_parquet(SIDE_MAP_PATH)  # 欄位：_idx, Id

n_rows   = side.height
n_null   = side["Id"].null_count()
n_nn     = n_rows - n_null
n_unique = side.filter(pl.col("Id").is_not_null()).select(pl.col("Id").n_unique()).item()
dupes    = n_nn - n_unique

# 各 Id 出現次數（只看非空）
counts = (
    side.filter(pl.col("Id").is_not_null())
        .group_by("Id")
        .len()
        .sort("len", descending=True)
)

max_count = counts.select(pl.col("len").max()).item() if counts.height else 0

print(f"rows(total): {n_rows:,}")
print(f"Id null    : {n_null:,}")
print(f"Id non-null: {n_nn:,}")
print(f"Id unique  : {n_unique:,}")
print(f"Id duplicates (non-null - unique): {dupes:,}")
print(f"max occurrences of a single Id   : {max_count:,}")

# 列出前 10 個重複 Id 的樣子
top_dupes = counts.filter(pl.col("len") > 1).head(10)
if top_dupes.height:
    sample = (
        side.join(top_dupes.select("Id"), on="Id", how="inner")
            .group_by("Id")
            .agg([
                pl.len().alias("n"),
                pl.col("_idx").head(5).alias("sample_idx")  # 每個 Id 範例的前 5 個 _idx
            ])
            .sort("n", descending=True)
    )
    print("\nTop duplicate Ids (n, sample_idx):")
    print(sample)

# 簡答你的問題：
if max_count == n_rows and n_rows > 0:
    print("\n⚠️ 注意：max_count == len(side)。代表所有列都被指到同一個 Id，這幾乎可以確定是簽名撞在一起了。")
elif max_count > 1:
    print("\nℹ️ 有些 Id 對到多於一列（簽名不夠唯一）；理想情況是 max_count == 1。")
else:
    print("\n✅ 每個 Id 只對到一列（max_count == 1），這是理想狀態。")


🔑 signature columns: ['totalPrice', 'taxes', 'legs0_departureAt', 'legs0_arrivalAt', 'legs0_segments0_flightNumber', 'legs0_segments0_marketingCarrier_code', 'legs0_segments0_departureFrom_airport_iata', 'legs0_segments0_arrivalTo_airport_iata', 'legs1_departureAt', 'legs1_arrivalAt', 'legs1_segments0_flightNumber', 'legs1_segments0_marketingCarrier_code', 'legs1_segments0_departureFrom_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs0_segments1_flightNumber', 'legs1_segments1_flightNumber', 'corporateTariffCode', 'legs0_segments1_marketingCarrier_code', 'legs0_segments2_flightNumber', 'legs1_segments2_flightNumber', 'legs0_segments1_duration']
🗺️ mapping rows (with position): 18,145,372
🧭 side map (multiplicity-aware) written → data/_id_side_map.parquet
rows(total): 18,151,423
Id null    : 6,051
Id non-null: 18,145,372
Id unique  : 18,145,372
Id duplicates (non-null - unique): 0
max occurrences of a single Id   : 1

✅ 每個 Id 只對到一列（max_count == 1），這是理想狀態。


In [4]:
side


_idx,Id,selected
u32,i64,i64
1701713,6061095,0
1701735,1567043,0
1701791,6654313,0
1701836,17361322,0
1701842,17361328,0
…,…,…
18056790,4475703,0
18056858,14366824,0
18056864,14366830,0


In [ ]:
import polars as pl
import os

TRAIN_FILLED_PATH = "data/train.parquet"
DF_PATH           = "data/train_all_columns.parquet"   # 很大的 df
OUT_PATH          = "data/train_all_columns_with_id.parquet"
SIDE_MAP_PATH     = "data/_id_side_map.parquet"        # 臨時側檔：(_idx, Id)

# ========= 1) 讀 train_filled（較小），決定可用的簽名欄 =========
train_filled = pl.read_parquet(TRAIN_FILLED_PATH)
df_schema = pl.read_parquet(DF_PATH, n_rows=0).schema
df_cols   = set(df_schema.keys())

# 一組「穩定的」候選欄位（盡量用第一段航段與第一個 segment；避免 duration 這類容易格式化不同的欄位）
SIGNATURE_CANDIDATES  = [
    "totalPrice", 'taxes',
    "legs0_departureAt", "legs0_arrivalAt",
    "legs0_segments0_flightNumber",
    "legs0_segments0_marketingCarrier_code",
    "legs0_segments0_departureFrom_airport_iata",
    "legs0_segments0_arrivalTo_airport_iata",
    # 若是來回多半也能用到（存在就會被選用）
    "legs1_departureAt", "legs1_arrivalAt",
    "legs1_segments0_flightNumber",
    "legs1_segments0_marketingCarrier_code",
    "legs1_segments0_departureFrom_airport_iata",
    "legs1_segments0_arrivalTo_airport_iata",
    'legs0_segments1_flightNumber', 'legs1_segments1_flightNumber',
    # 'legs1_segments0_seatsAvailable','legs0_segments0_seatsAvailable',
    # 'legs1_segments1_seatsAvailable',#'legs0_segments1_baggageAllowance_quantity','legs0_segments1_seatsAvailable' , "miniRules0_monetaryAmount", "miniRules1_monetaryAmount","miniRules0_statusInfos"
    # 'legs1_segments0_baggageAllowance_quantity', 
    'corporateTariffCode',
    # 'legs0_segments0_baggageAllowance_quantity',
    'legs0_segments1_marketingCarrier_code', 'legs0_segments2_flightNumber',
    # 'legs1_segments1_baggageAllowance_quantity', #'legs0_segments1_baggageAllowance_quantity',
    # 'legs1_segments2_baggageAllowance_quantity', #'legs0_segments2_baggageAllowance_quantity',
    'legs1_segments2_flightNumber', #'legs1_segments0_baggageAllowance_weightMeasurementType', 
    # 'legs0_segments2_seatsAvailable',
    'legs0_segments1_duration',
]

# 只取雙方都存在的
sig_cols = [c for c in SIGNATURE_CANDIDATES if c in train_filled.columns and c in df_cols]

if len(sig_cols) < 4:
    # 後備：用交集裡的前 N 個（避免欄太少）
    common_cols = [c for c in train_filled.columns if c in df_cols and c not in {"Id", "selected", "ranker_id"}]
    sig_cols = sorted(common_cols)[:8]  # 至少湊一些
print(f"🔑 signature columns: {sig_cols}")

# ========= 2) 簽名正規化（修正版）=========
def normalized_sig_exprs(cols):
    """
    - departureAt/arrivalAt：擷取到分鐘 "YYYY-MM-DDTHH:MM"
    - duration：從 "HH:MM:SS" 轉總分鐘（整數字串）
    - 其他：轉字串並去掉尾端無資訊的 ".0"
    - 缺值 → "∅"
    """
    exprs = []
    for c in cols:
        e = pl.col(c)

        if c.endswith("departureAt") or c.endswith("arrivalAt"):
            e = e.cast(pl.Utf8).str.extract(r"^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})", 1)

        elif c.endswith("duration"):
            s  = e.cast(pl.Utf8)
            hh = s.str.extract(r"^(\d+):", 1).cast(pl.Int64, strict=False)
            mm = s.str.extract(r"^\d+:(\d{2})", 1).cast(pl.Int64, strict=False)
            total_min = (hh * pl.lit(60) + mm)
            e = pl.when(hh.is_not_null() & mm.is_not_null()) \
                 .then(total_min.cast(pl.Int64).cast(pl.Utf8)) \
                 .otherwise(None)

        else:
            e = e.cast(pl.Utf8).str.replace(r"\.0+$", "")  # "9.0" -> "9"

        exprs.append(pl.coalesce([e, pl.lit("∅")]).alias(f"__sig__{c}"))
    return exprs

def signature_expr(cols):
    return pl.concat_str(
        [pl.col("ranker_id").cast(pl.Utf8)] + [pl.col(f"__sig__{c}") for c in cols],
        separator="|", ignore_nulls=False
    ).alias("_sig")


# ========= 3) (ranker_id, _sig, pos) -> Id 對照表（train_filled 端；用 rank 產生 pos）=========
train_sig = (
    train_filled.lazy()
    .with_row_index("_rid")                                  # 穩定順序
    .select(["ranker_id","selected", "Id", "_rid"] + sig_cols)
    .with_columns(normalized_sig_exprs(sig_cols))
    .with_columns(signature_expr(sig_cols))                  # 產生 _sig
    .with_columns([
        # 在每個 (ranker_id, _sig) 內，依 _rid 做 ordinal rank，得到 1..n，再 -1 變 0..n-1
        (pl.col("_rid")
           .rank(method="ordinal")
           .over(["_sig"])
           .cast(pl.Int64) - 1
        ).alias("pos")
    ])
    .select(["ranker_id", "_sig", "pos", "Id","selected"])
    .collect()
)

print(f"🗺️ mapping rows (with position): {train_sig.height:,}")


# ========= 4) 大表端也產生 (ranker_id, _sig, pos)，再 join =========
lf_small = pl.scan_parquet(DF_PATH).select(["ranker_id"] + sig_cols)

lf_side = (
    lf_small
    .with_row_index("_idx")                                  # 穩定順序
    .with_columns(normalized_sig_exprs(sig_cols))
    .with_columns(signature_expr(sig_cols))                  # 產生 _sig
    .with_columns([
        (pl.col("_idx")
           .rank(method="ordinal")
           .over(["_sig"])
           .cast(pl.Int64) - 1
        ).alias("pos")
    ])
    .join(train_sig.lazy(), on=["ranker_id", "_sig", "pos"], how="left")
    .select(["_idx", "Id", "selected"])
)



os.makedirs(os.path.dirname(SIDE_MAP_PATH), exist_ok=True)
lf_side.sink_parquet(SIDE_MAP_PATH, compression="zstd")
print(f"🧭 side map (multiplicity-aware) written → {SIDE_MAP_PATH}")

# （可選）看一下配對率
import polars as pl

side = pl.read_parquet(SIDE_MAP_PATH)  # 欄位：_idx, Id

n_rows   = side.height
n_null   = side["Id"].null_count()
n_nn     = n_rows - n_null
n_unique = side.filter(pl.col("Id").is_not_null()).select(pl.col("Id").n_unique()).item()
dupes    = n_nn - n_unique

# 各 Id 出現次數（只看非空）
counts = (
    side.filter(pl.col("Id").is_not_null())
        .group_by("Id")
        .len()
        .sort("len", descending=True)
)

max_count = counts.select(pl.col("len").max()).item() if counts.height else 0

print(f"rows(total): {n_rows:,}")
print(f"Id null    : {n_null:,}")
print(f"Id non-null: {n_nn:,}")
print(f"Id unique  : {n_unique:,}")
print(f"Id duplicates (non-null - unique): {dupes:,}")
print(f"max occurrences of a single Id   : {max_count:,}")

# 列出前 10 個重複 Id 的樣子
top_dupes = counts.filter(pl.col("len") > 1).head(10)
if top_dupes.height:
    sample = (
        side.join(top_dupes.select("Id"), on="Id", how="inner")
            .group_by("Id")
            .agg([
                pl.len().alias("n"),
                pl.col("_idx").head(5).alias("sample_idx")  # 每個 Id 範例的前 5 個 _idx
            ])
            .sort("n", descending=True)
    )
    print("\nTop duplicate Ids (n, sample_idx):")
    print(sample)

# 簡答你的問題：
if max_count == n_rows and n_rows > 0:
    print("\n⚠️ 注意：max_count == len(side)。代表所有列都被指到同一個 Id，這幾乎可以確定是簽名撞在一起了。")
elif max_count > 1:
    print("\nℹ️ 有些 Id 對到多於一列（簽名不夠唯一）；理想情況是 max_count == 1。")
else:
    print("\n✅ 每個 Id 只對到一列（max_count == 1），這是理想狀態。")


✅ train_filled 中 selected 筆數：105,539


ColumnNotFoundError: unable to find column "Iselected"; valid columns: ["_idx", "ranker_id", "totalPrice", "taxes", "legs0_departureAt", "legs0_arrivalAt", "legs0_segments0_flightNumber", "legs0_segments0_marketingCarrier_code", "legs0_segments0_departureFrom_airport_iata", "legs0_segments0_arrivalTo_airport_iata", "legs1_departureAt", "legs1_arrivalAt", "legs1_segments0_flightNumber", "legs1_segments0_marketingCarrier_code", "legs1_segments0_departureFrom_airport_iata", "legs1_segments0_arrivalTo_airport_iata", "legs0_segments1_flightNumber", "legs1_segments1_flightNumber", "corporateTariffCode", "legs0_segments1_marketingCarrier_code", "legs0_segments2_flightNumber", "legs1_segments2_flightNumber", "legs0_segments1_duration", "__sig__totalPrice", "__sig__taxes", "__sig__legs0_departureAt", "__sig__legs0_arrivalAt", "__sig__legs0_segments0_flightNumber", "__sig__legs0_segments0_marketingCarrier_code", "__sig__legs0_segments0_departureFrom_airport_iata", "__sig__legs0_segments0_arrival

In [20]:
# ========= 6) 分析 side 中重複的 Id，按 ranker_id 統計 =========
import polars as pl
# import os

# REPORT_DIR = "data/signature_diagnostics"
# os.makedirs(REPORT_DIR, exist_ok=True)

# 6.1 讀 side，找出「重複的 Id」
side = pl.read_parquet(SIDE_MAP_PATH)

dup_ids = (
    side
    .filter(pl.col("Id").is_not_null())
    .group_by("Id")
    .len()
    .rename({"len": "n_rows"})
    .filter(pl.col("n_rows") > 1)         # 僅保留在 side 中出現超過 1 次的 Id
)

if dup_ids.height == 0:
    print("✅ side 中沒有重複的 Id。")
else:
    print(f"🔁 side 中有 {dup_ids.height} 個 Id 是重複的。")

    # 6.2 只保留 side 裡那些重複 Id 的列，並接回 DF_PATH 取得 ranker_id（用 _idx 對應）
    side_dups = side.join(dup_ids.select("Id"), on="Id", how="inner")

    df_rankers = (
        pl.scan_parquet(DF_PATH)
        .select(["ranker_id"])
        .with_row_index("_idx")
        .filter(pl.col("_idx").is_in(side_dups.select("_idx").to_series().to_list()))
        .collect()
    )

    side_dups = side_dups.join(df_rankers, on="_idx", how="left")

    # 6.3 逐 pair 統計：每個 (ranker_id, Id) 在 side 出現幾次
    per_pair = (
        side_dups
        .group_by(["ranker_id", "Id"])
        .len()
        .rename({"len": "n_within_ranker"})
        .sort(["n_within_ranker"], descending=True)
    )

    # 6.4 只看「在同一個 ranker_id 內真的重複（n>1）」的 Id，按 ranker 彙總
    per_ranker_dup_only = (
        per_pair
        .filter(pl.col("n_within_ranker") > 1)
        .group_by("ranker_id")
        .agg([
            pl.len().alias("num_ids_duplicated_within_ranker"),  # 這個 ranker 中，有多少個 Id 是重複的
            pl.col("n_within_ranker").sum().alias("total_rows_of_those_ids"),
            pl.col("n_within_ranker").max().alias("max_occurrences_for_a_single_id"),
        ])
        .sort("num_ids_duplicated_within_ranker", descending=True)
    )

    # 6.5 每個 ranker 的重複統計（以 pair 表為基礎）
    per_ranker_all = (
        side_dups
        .group_by(["ranker_id", "Id"])
        .len()
        .rename({"len": "n_within_ranker"})
        .group_by("ranker_id")
        .agg([
            # 只把重複 (n>1) 的那些行數加總
            pl.when(pl.col("n_within_ranker") > 1)
            .then(pl.col("n_within_ranker"))
            .otherwise(0)
            .sum()
            .alias("rows_with_duplicated_ids"),

            # 重複的「唯一 Id 數量」= 計算 (n>1) 的個數
            (pl.col("n_within_ranker") > 1)
            .cast(pl.Int8)
            .sum()
            .alias("unique_duplicated_ids"),

            # 該 ranker 內單一 Id 的最大重複次數（參考用）
            pl.col("n_within_ranker").max().alias("max_occurrences_for_a_single_id"),
        ])
        .sort("rows_with_duplicated_ids", descending=True)
    )

    max_val = per_pair.select(pl.col("n_within_ranker").max()).item()
    print(f"🔍 n_within_ranker 的全域最大值: {max_val}")

    # # 6.6 輸出 / 印前幾筆
    # out_pairs = os.path.join(REPORT_DIR, "duplicated_ids_by_ranker_pair.parquet")
    # out_ranker = os.path.join(REPORT_DIR, "duplicated_ids_by_ranker_summary.parquet")

    # per_pair.write_parquet(out_pairs, compression="zstd")
    # per_ranker_dup_only.write_parquet(out_ranker, compression="zstd")

    print("\n📊 (ranker_id, Id) 出現次數（Top 20）:")
    print(per_pair.head(20))

    print("\n🧮 每個 ranker_id 內真正重複的 Id 統計（n>1）：")
    print(per_ranker_dup_only.head(20))

    # print(f"\n💾 已輸出明細: {out_pairs}")
    # print(f"💾 已輸出彙總: {out_ranker}")


🔁 side 中有 12588 個 Id 是重複的。
🔍 n_within_ranker 的全域最大值: 2

📊 (ranker_id, Id) 出現次數（Top 20）:
shape: (20, 3)
┌─────────────────────────────────┬─────────┬─────────────────┐
│ ranker_id                       ┆ Id      ┆ n_within_ranker │
│ ---                             ┆ ---     ┆ ---             │
│ str                             ┆ i64     ┆ u32             │
╞═════════════════════════════════╪═════════╪═════════════════╡
│ 4cd5f38aff504926902d41bd33044a… ┆ 8345031 ┆ 2               │
│ 92e4a81814694808974f778d14623f… ┆ 1225472 ┆ 2               │
│ 21969f8f6ab24596be0691cefc584b… ┆ 8910330 ┆ 2               │
│ b17a8e60dd45423083be86a5ef7091… ┆ 1955440 ┆ 2               │
│ f6c99c34f24844beb96b446d3dca2e… ┆ 1376302 ┆ 2               │
│ …                               ┆ …       ┆ …               │
│ 536f0381b08b49b4b4c4e8f6564f82… ┆ 76521   ┆ 2               │
│ 0823ffe26e454eae82ce1853892051… ┆ 4503599 ┆ 2               │
│ 1de7bcae8eb3499295b92a4c7bfe24… ┆ 5324747 ┆ 2               │
│

# see duplicate

In [44]:
# ========= 5) 診斷 side 中 Id=NULL：對照 DF 與 train_filled =========
import polars as pl
import os

DIAG_DIR = "data/signature_diagnostics"
os.makedirs(DIAG_DIR, exist_ok=True)

MAX_SHOW = 30  # console 印出的筆數上限

# 5.1 讀 side，抓 NULL 的 _idx
side = pl.read_parquet(SIDE_MAP_PATH)
null_idx = (
    side.filter(pl.col("Id").is_null())
        .select("_idx")
        .to_series()
        .to_list()
)

print(f"\n🔎 side 中 Id=NULL 筆數：{len(null_idx)}")
if not null_idx:
    print("✅ 沒有需要診斷的 NULL。")
else:
    # 5.2 DF 端：取出這些 _idx，計算 _sig 與兩種 pos（over(['_sig']) 與 over(['ranker_id','_sig'])）
    df_null = (
        pl.scan_parquet(DF_PATH)
          .select(["ranker_id"] + sig_cols)
          .with_row_index("_idx")
          .filter(pl.col("_idx").is_in(null_idx))
          .with_columns(normalized_sig_exprs(sig_cols))
          .with_columns(signature_expr(sig_cols))
          .with_columns([
              # 你現在使用的做法（僅以 _sig 分組）
              (pl.col("_idx").rank(method="ordinal").over(["_sig"]).cast(pl.Int64) - 1).alias("pos_over_sig"),
              # 建議的穩定做法（同時以 ranker_id 與 _sig 分組）
              (pl.col("_idx").rank(method="ordinal").over(["ranker_id","_sig"]).cast(pl.Int64) - 1).alias("pos_over_rsig"),
          ])
          .select(["_idx","ranker_id","_sig","pos_over_sig","pos_over_rsig"])
          .collect()
    )

    # 5.3 train 端：同樣計算 _sig 與兩種 pos，並且算每個群在 train 的筆數
    train_norm = (
        train_filled.lazy()
        .with_row_index("_rid")
        .select(["ranker_id","Id","_rid"] + sig_cols)
        .with_columns(normalized_sig_exprs(sig_cols))
        .with_columns(signature_expr(sig_cols))
        .with_columns([
            (pl.col("_rid").rank(method="ordinal").over(["_sig"]).cast(pl.Int64) - 1).alias("pos_over_sig"),
            (pl.col("_rid").rank(method="ordinal").over(["ranker_id","_sig"]).cast(pl.Int64) - 1).alias("pos_over_rsig"),
        ])
        .collect()
    )

    train_counts_sig = (
        train_norm.group_by(["ranker_id","_sig"])
                  .len()
                  .rename({"len":"train_n"})
    )

    # 5.4 兩種對齊方式分別 join，看看是否能在 train 找到 Id
    # A) 你現在 side 的方式：on ["ranker_id","_sig","pos_over_sig"]
    matched_sig = (
        df_null
        .join(train_norm.select(["ranker_id","_sig","pos_over_sig","Id"]),
              on=["ranker_id","_sig","pos_over_sig"], how="left")
        .rename({"Id":"Id_by_pos_over_sig"})
    )

    # B) 建議的穩定方式：on ["ranker_id","_sig","pos_over_rsig"]
    matched_rsig = (
        df_null
        .join(train_norm.select(["ranker_id","_sig","pos_over_rsig","Id"]),
              on=["ranker_id","_sig","pos_over_rsig"], how="left")
        .rename({"Id":"Id_by_pos_over_rsig"})
    )

    diag = (
        matched_sig
        .join(matched_rsig.select(["_idx","Id_by_pos_over_rsig"]), on="_idx", how="left")
        .join(train_counts_sig, on=["ranker_id","_sig"], how="left")
        .with_columns([
            pl.col("train_n").fill_null(0).alias("train_n"),
            (pl.col("Id_by_pos_over_sig").is_null()).alias("null_by_sig"),
            (pl.col("Id_by_pos_over_rsig").is_null()).alias("null_by_rsig"),
            (pl.col("pos_over_sig") >= pl.col("train_n")).alias("pos_exceeds_train_n"),
            (pl.col("train_n") == 0).alias("group_missing_in_train"),
            pl.when(pl.col("group_missing_in_train"))
              .then(pl.lit("missing_group_in_train"))
              .when(~pl.col("group_missing_in_train") & pl.col("pos_exceeds_train_n"))
              .then(pl.lit("df_more_than_train_in_group"))
              .when(pl.col("Id_by_pos_over_sig").is_null() & pl.col("Id_by_pos_over_rsig").is_not_null())
              .then(pl.lit("wrong_window_use_ranker_id"))
              .otherwise(pl.lit("other"))
              .alias("null_reason")
        ])
        .select([
            "_idx","ranker_id","_sig","train_n","pos_over_sig","pos_over_rsig",
            "Id_by_pos_over_sig","Id_by_pos_over_rsig","null_reason"
        ])
        .sort(["null_reason","ranker_id","_sig","pos_over_sig"])
    )

    # # 5.5 輸出與摘要
    # out_path = os.path.join(DIAG_DIR, "side_null_diagnosis.parquet")
    # diag.write_parquet(out_path, compression="zstd")
    # print(f"💾 已輸出診斷表：{out_path}")

    print("\n📊 NULL 原因分佈（前幾個）：")
    reason_counts = diag.group_by("null_reason").len().sort("len", descending=True)
    print(reason_counts)

    print("\n🧾 範例（前幾筆）：")
    print(diag.head(MAX_SHOW))



🔎 side 中 Id=NULL 筆數：6051


ColumnNotFoundError: unable to find column "group_missing_in_train"; valid columns: ["_idx", "ranker_id", "_sig", "pos_over_sig", "pos_over_rsig", "Id_by_pos_over_sig", "Id_by_pos_over_rsig", "train_n"]

In [3]:
# ========= 只把 lf_side 裡 Id=NULL 的位置抓回 DF_PATH 看原始列 =========
import polars as pl
import os

MAX_SHOW = 50  # 一次印幾筆，避免 console 爆量；需要可調大或存檔看

# 1) 讀 side map，抓出所有 Id=NULL 的 _idx
side = pl.read_parquet(SIDE_MAP_PATH)
null_idx = (
    side.filter(pl.col("Id").is_null())
        .select("_idx")
        .to_series()
        .to_list()
)

print(f"🔎 找到 {len(null_idx)} 筆 Id=NULL。")
if not null_idx:
    print("✅ 沒有需要檢視的 NULL。")
else:
    # 2) 回 DF_PATH 依 _idx 抽樣原始列（只取 ranker_id + 簽名欄位，夠看差異）
    df_null_rows = (
        pl.scan_parquet(DF_PATH)
          .select(["ranker_id"] + sig_cols)
          .with_row_index("_idx")                       # 這個 _idx 就是你在 lf_side 用的「iloc」
          .filter(pl.col("_idx").is_in(null_idx))
          .collect()
          .sort("_idx")
    )

    # 3) 印出前 MAX_SHOW 筆看看
    print("\n🧾 這些是 DF_PATH 中（原始值）的對應列：")
    print(df_null_rows.head(MAX_SHOW))


🔎 找到 6051 筆 Id=NULL。

🧾 這些是 DF_PATH 中（原始值）的對應列：
shape: (50, 23)
┌────────┬────────────┬────────────┬─────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ _idx   ┆ ranker_id  ┆ totalPrice ┆ taxes   ┆ … ┆ legs0_segm ┆ legs0_segm ┆ legs1_seg ┆ legs0_seg │
│ ---    ┆ ---        ┆ ---        ┆ ---     ┆   ┆ ents1_mark ┆ ents2_flig ┆ ments2_fl ┆ ments1_du │
│ u32    ┆ str        ┆ f64        ┆ f64     ┆   ┆ etingCarri ┆ htNumber   ┆ ightNumbe ┆ ration    │
│        ┆            ┆            ┆         ┆   ┆ …          ┆ ---        ┆ r         ┆ ---       │
│        ┆            ┆            ┆         ┆   ┆ ---        ┆ str        ┆ ---       ┆ str       │
│        ┆            ┆            ┆         ┆   ┆ str        ┆            ┆ str       ┆           │
╞════════╪════════════╪════════════╪═════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 1609   ┆ 15fddfb708 ┆ 118761.0   ┆ 13571.0 ┆ … ┆ EY         ┆ null       ┆ null      ┆ 04:15:00  │
│        ┆ 664a019b35 ┆    

In [42]:
null_idx

[388655,
 391694,
 392215,
 391719,
 392242,
 391469,
 388647,
 391686,
 391599,
 392277,
 923440,
 923444,
 927611,
 923400,
 923452,
 927631,
 927635,
 927639,
 923448,
 927619,
 927623,
 1973734,
 1973726,
 1973738,
 1973742,
 1441585,
 1441508,
 1441475,
 1441493,
 1441535,
 1441553,
 1441530,
 1441589,
 1441413,
 1441434,
 1441570,
 1441388,
 1441496,
 1441503,
 1441463,
 1441480,
 1441395,
 1441562,
 1441460,
 1441468,
 1441385,
 2094321,
 2094361,
 2094330,
 2094370,
 1805725,
 1805723,
 113588,
 113739,
 113599,
 113696,
 113799,
 113630,
 1592077,
 1592207,
 1606256,
 1592118,
 1592197,
 1592277,
 1592300,
 1592393,
 1592221,
 1592263,
 1592227,
 1592271,
 1592312,
 1592426,
 1605971,
 1606034,
 1592109,
 1592351,
 1606230,
 1592134,
 1591833,
 1592233,
 1606286,
 1591863,
 1592059,
 1592450,
 1606318,
 1606356,
 1591801,
 1591918,
 1592026,
 1591958,
 1592251,
 1591973,
 1592215,
 1592306,
 1605966,
 1606252,
 1591893,
 1591983,
 1592050,
 1592158,
 1592173,
 1592245,
 159225

In [4]:
df_null_rows['ranker_id'][0]

'15fddfb708664a019b352e20f063deec'

In [25]:
# ========= 篩出特定 ranker_id & flightNumber 在 train_filled 的資料 =========
target_rid = "15fddfb708664a019b352e20f063deec"

target_rows = (
    train_filled
    .filter(pl.col("ranker_id") == target_rid)
    # .filter(pl.col("legs0_segments0_flightNumber").is_in(["2601"]))
    .filter(pl.col("totalPrice") == 118761.0)
    # .filter(pl.col("taxes") == 1513.0)


)

print(f"\n🎯 train_filled 中 ranker_id={target_rid} 且 flightNumber 在指定列表的資料（共 {target_rows.height} 列）:")

# 顯示正規化後的簽名欄位

train_select=target_rows.with_columns(normalized_sig_exprs(sig_cols)).with_columns(signature_expr(sig_cols))
train_select



🎯 train_filled 中 ranker_id=15fddfb708664a019b352e20f063deec 且 flightNumber 在指定列表的資料（共 2 列）:


Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__,__sig__totalPrice,__sig__taxes,__sig__legs0_departureAt,__sig__legs0_arrivalAt,__sig__legs0_segments0_flightNumber,__sig__legs0_segments0_marketingCarrier_code,__sig__legs0_segments0_departureFrom_airport_iata,__sig__legs0_segments0_arrivalTo_airport_iata,__sig__legs1_departureAt,__sig__legs1_arrivalAt,__sig__legs1_segments0_flightNumber,__sig__legs1_segments0_marketingCarrier_code,__sig__legs1_segments0_departureFrom_airport_iata,__sig__legs1_segments0_arrivalTo_airport_iata,__sig__legs0_segments1_flightNumber,__sig__legs1_segments1_flightNumber,__sig__corporateTariffCode,__sig__legs0_segments1_marketingCarrier_code,__sig__legs0_segments2_flightNumber,__sig__legs1_segments2_flightNumber,__sig__legs0_segments1_duration,_sig
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2911205,true,43648,null,null,36,false,false,"""2024-07-29T20:05:00""","""2024-07-28T23:35:00""","""18:00:00""","""789""","""AUH""","""AUH""",40.0,1.0,1.0,"""SVO""","""05:30:00""","""66""","""EY""","""EY""",9.0,"""32Q""","""BLR""","""BLR""",40.0,1.0,1.0,"""AUH""","""04:15:00""","""238""","""EY""","""EY""",9.0,null,null,…,1.0,0.0,0.0,1.0,1.0,1,1750257,"""15fddfb708664a019b352e20f063de…",2024-06-27 12:31:24,"""MOWBLR""",true,13571.0,118761.0,0,2911205,"""118761""","""13571""","""2024-07-28T23:35""","""2024-07-29T20:05""","""66""","""EY""","""SVO""","""AUH""","""∅""","""∅""","""∅""","""∅""","""∅""","""∅""","""238""","""∅""","""∅""","""EY""","""∅""","""∅""","""255""","""15fddfb708664a019b352e20f063de…"
2911229,true,43648,null,null,36,false,false,"""2024-07-30T03:05:00""","""2024-07-28T23:35:00""","""1.01:00:00""","""789""","""AUH""","""AUH""",40.0,1.0,1.0,"""SVO""","""05:30:00""","""66""","""EY""","""EY""",9.0,"""789""","""BLR""","""BLR""",40.0,1.0,1.0,"""AUH""","""04:15:00""","""216""","""EY""","""EY""",9.0,null,null,…,1.0,0.0,0.0,1.0,1.0,1,1750257,"""15fddfb708664a019b352e20f063de…",2024-06-27 12:31:24,"""MOWBLR""",true,13571.0,118761.0,0,2911229,"""118761""","""13571""","""2024-07-28T23:35""","""2024-07-30T03:05""","""66""","""EY""","""SVO""","""AUH""","""∅""","""∅""","""∅""","""∅""","""∅""","""∅""","""216""","""∅""","""∅""","""EY""","""∅""","""∅""","""255""","""15fddfb708664a019b352e20f063de…"


In [28]:
target_rows['_sig'][0]

'15fddfb708664a019b352e20f063deec|118761|13571|2024-07-28T23:35|2024-07-29T20:05|66|EY|SVO|AUH|∅|∅|∅|∅|∅|∅|238|∅|∅|EY|∅|∅|255'

In [9]:
train_select['_sig'][0]

'15fddfb708664a019b352e20f063deec|118761|13571|2024-07-28T23:35|2024-07-29T20:05|66|EY|SVO|AUH|∅|∅|∅|∅|∅|∅|238|∅|∅|EY|∅|∅|255'

In [11]:
df_select['_sig'][0]

'15fddfb708664a019b352e20f063deec|118761|13571|2024-07-28T23:35|2024-07-29T20:05|66|EY|SVO|AUH|∅|∅|∅|∅|∅|∅|238|∅|∅|EY|∅|∅|255'

In [41]:
# ========= 篩出特定 ranker_id & flightNumber 在 train_filled 的資料 =========

target_rows = (
    df_null_rows
    .filter(pl.col("ranker_id") == target_rid)
    # .filter(pl.col("legs0_segments0_flightNumber").is_in(["2601"]))
    .filter(pl.col("totalPrice") == 118761.0)

)



print(f"\n🎯 train_filled 中 ranker_id={target_rid} 且 flightNumber 在指定列表的資料（共 {target_rows.height} 列）:")

# 顯示正規化後的簽名欄位

df_select = target_rows.with_columns(normalized_sig_exprs(sig_cols)).with_columns(signature_expr(sig_cols))

df_select


🎯 train_filled 中 ranker_id=15fddfb708664a019b352e20f063deec 且 flightNumber 在指定列表的資料（共 2 列）:


_idx,ranker_id,totalPrice,taxes,legs0_departureAt,legs0_arrivalAt,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_departureFrom_airport_iata,legs0_segments0_arrivalTo_airport_iata,legs1_departureAt,legs1_arrivalAt,legs1_segments0_flightNumber,legs1_segments0_marketingCarrier_code,legs1_segments0_departureFrom_airport_iata,legs1_segments0_arrivalTo_airport_iata,legs0_segments1_flightNumber,legs1_segments1_flightNumber,corporateTariffCode,legs0_segments1_marketingCarrier_code,legs0_segments2_flightNumber,legs1_segments2_flightNumber,legs0_segments1_duration,__sig__totalPrice,__sig__taxes,__sig__legs0_departureAt,__sig__legs0_arrivalAt,__sig__legs0_segments0_flightNumber,__sig__legs0_segments0_marketingCarrier_code,__sig__legs0_segments0_departureFrom_airport_iata,__sig__legs0_segments0_arrivalTo_airport_iata,__sig__legs1_departureAt,__sig__legs1_arrivalAt,__sig__legs1_segments0_flightNumber,__sig__legs1_segments0_marketingCarrier_code,__sig__legs1_segments0_departureFrom_airport_iata,__sig__legs1_segments0_arrivalTo_airport_iata,__sig__legs0_segments1_flightNumber,__sig__legs1_segments1_flightNumber,__sig__corporateTariffCode,__sig__legs0_segments1_marketingCarrier_code,__sig__legs0_segments2_flightNumber,__sig__legs1_segments2_flightNumber,__sig__legs0_segments1_duration,_sig
u32,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1609,"""15fddfb708664a019b352e20f063de…",118761.0,13571.0,"""2024-07-28T23:35:00""","""2024-07-29T20:05:00""","""66""","""EY""","""SVO""","""AUH""",null,null,null,null,null,null,"""238""",null,null,"""EY""",null,null,"""04:15:00""","""118761""","""13571""","""2024-07-28T23:35""","""2024-07-29T20:05""","""66""","""EY""","""SVO""","""AUH""","""∅""","""∅""","""∅""","""∅""","""∅""","""∅""","""238""","""∅""","""∅""","""EY""","""∅""","""∅""","""255""","""15fddfb708664a019b352e20f063de…"
1635,"""15fddfb708664a019b352e20f063de…",118761.0,13571.0,"""2024-07-28T23:35:00""","""2024-07-30T03:05:00""","""66""","""EY""","""SVO""","""AUH""",null,null,null,null,null,null,"""216""",null,null,"""EY""",null,null,"""04:15:00""","""118761""","""13571""","""2024-07-28T23:35""","""2024-07-30T03:05""","""66""","""EY""","""SVO""","""AUH""","""∅""","""∅""","""∅""","""∅""","""∅""","""∅""","""216""","""∅""","""∅""","""EY""","""∅""","""∅""","""255""","""15fddfb708664a019b352e20f063de…"


In [ ]:
df_select['Id']

ColumnNotFoundError: "Id" not found

In [5]:
import polars as pl
import os

DF_PATH       = "data/test_all_columns.parquet"   # 大表
SIDE_MAP_PATH = "data/_id_side_map.parquet"        # 只有 _idx, Id
OUT_PATH      = "data/data_from_json/test/test_all_columns_with_id.parquet"

# 以串流方式讀兩個 parquet
lf_big  = pl.scan_parquet(DF_PATH).with_row_index("_idx")
lf_side = pl.scan_parquet(SIDE_MAP_PATH)

# 保險起見，對齊 _idx 的 dtype（避免型別不合）
lf_big  = lf_big.with_columns(pl.col("_idx").cast(pl.UInt64))
lf_side = lf_side.with_columns(pl.col("_idx").cast(pl.UInt64))

# 左連接，把 Id 併回去；不想保留 _idx 就 drop 掉
lf_joined = (
    lf_big
    .join(lf_side, on="_idx", how="left")
    .drop("_idx")    # 如果想留下 _idx 就把這行註解掉
)

# 輸出
os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
lf_joined.sink_parquet(OUT_PATH, compression="zstd")
print("✅ saved to:", OUT_PATH)


✅ saved to: data/data_from_json/test/test_all_columns_with_id.parquet


In [ ]:
import polars as pl
import os

ORIG_TEST_PATH   = "data/test.parquet"  # 原本的 test
ENRICHED_PATH    = "data/data_from_json/test/test_all_columns_with_id.parquet"  # 有較完整 features
OUT_MERGED_PATH  = "data/data_from_json/test/test_merged.parquet"

# ========== 讀兩邊（Lazy）==========
lf_orig = pl.scan_parquet(ORIG_TEST_PATH)
lf_enr  = pl.scan_parquet(ENRICHED_PATH).unique(subset=["Id"], keep="first")  # 保每個 Id 一列

# （可選）如果你不想帶入 pandas 產生的索引欄，可先過濾掉
DROP_NOISE = {"__index_level_0__"}  # 依實際情況再加
lf_orig = lf_orig.select([c for c in lf_orig.columns if c not in DROP_NOISE])
lf_enr  = lf_enr.select([c for c in lf_enr.columns if c not in DROP_NOISE])

# ========== 欄位集合 ==========
orig_cols = set(lf_orig.columns)
enr_cols  = set(lf_enr.columns)

missing_in_enr  = sorted(orig_cols - enr_cols)  # enriched 缺 → 要用 orig 真值補
missing_in_orig = sorted(enr_cols - orig_cols)  # orig 缺   → 之後補 null

# ========== 先把 missing_in_enr 的真值補到 enriched ==========
if missing_in_enr:
    lf_enr = lf_enr.join(
        lf_orig.select(["Id"] + missing_in_enr),
        on="Id",
        how="left",
    )

# 重新取 schema（因為剛剛補過欄位）
orig_schema = lf_orig.schema        # dict: {col: dtype}
enr_schema  = lf_enr.schema

# ========== 決定最終欄位順序與 dtype ==========
# 欄位順序：沿用原本 test 的順序，最後接上 enriched 才有的欄位
all_cols = lf_orig.columns + [c for c in lf_enr.columns if c not in lf_orig.columns]

# 最終 dtype 規則：
# - 共同欄位：以 orig 的 dtype 為準
# - 只在 orig：用 orig 的 dtype
# - 只在 enr ：用 enr  的 dtype
# - 萬一都取不到（理論上不會）→ Utf8
final_dtype: dict[str, pl.datatypes.DataType] = {}
for c in all_cols:
    if c in orig_schema:
        final_dtype[c] = orig_schema[c]
    elif c in enr_schema:
        final_dtype[c] = enr_schema[c]
    else:
        final_dtype[c] = pl.Utf8  # 後備

# ========== 讓兩邊補齊所有欄位（缺的補 null，且型別對齊）==========
def ensure_columns(lf: pl.LazyFrame, target_cols: list[str], target_dtype: dict[str, pl.DataType]) -> pl.LazyFrame:
    have = set(lf.columns)
    # 先補缺的欄位（null；dtype 指定成目標型別，免得 concat 時衝突）
    add_exprs = [pl.lit(None, dtype=target_dtype[c]).alias(c) for c in target_cols if c not in have]
    if add_exprs:
        lf = lf.with_columns(add_exprs)
    # 再把所有欄位 cast 成目標 dtype，並重排欄位順序
    cast_exprs = [pl.col(c).cast(target_dtype[c], strict=False) for c in target_cols]
    return lf.with_columns(cast_exprs).select(target_cols)

lf_orig_aligned = ensure_columns(lf_orig, all_cols, final_dtype)
lf_enr_aligned  = ensure_columns(lf_enr,  all_cols, final_dtype)

# ========== 覆蓋邏輯：相同 Id 用 enriched 覆蓋，orig 只保留 anti 的 ==========
lf_orig_keep = lf_orig_aligned.join(lf_enr_aligned.select(["Id"]), on="Id", how="anti")

lf_merged = pl.concat([lf_enr_aligned, lf_orig_keep], how="vertical")

# ========== 寫檔 ==========
os.makedirs(os.path.dirname(OUT_MERGED_PATH), exist_ok=True)
lf_merged.sink_parquet(OUT_MERGED_PATH, compression="zstd")
print("✅ saved to:", OUT_MERGED_PATH)


C:\Users\wani\AppData\Local\Temp\ipykernel_12304\1114533970.py:14: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  lf_orig = lf_orig.select([c for c in lf_orig.columns if c not in DROP_NOISE])
C:\Users\wani\AppData\Local\Temp\ipykernel_12304\1114533970.py:15: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  lf_enr  = lf_enr.select([c for c in lf_enr.columns if c not in DROP_NOISE])
C:\Users\wani\AppData\Local\Temp\ipykernel_12304\1114533970.py:18: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get th

In [ ]:
import polars as pl
import os

ORIG_TRAIN_PATH  = "data/train.parquet"  # 原本的 train（很大）
ENRICHED_PATH    = "data/data_from_json/train/train_all_columns_with_id.parquet"
OUT_ENRICHED_FIX = "data/data_from_json/train/train_merged.parquet"

# 讀兩邊（Lazy）
lf_orig = pl.scan_parquet(ORIG_TRAIN_PATH)
lf_enr  = pl.scan_parquet(ENRICHED_PATH)

# 1) 先把 ENRICHED 中 Id = NULL 的列 drop 掉
lf_enr = lf_enr.filter(pl.col("Id").is_not_null())

# （可選）去掉雜訊欄
DROP_NOISE = {"__index_level_0__"}  # 依實際資料調整
lf_orig = lf_orig.select([c for c in lf_orig.columns if c not in DROP_NOISE])
lf_enr  = lf_enr.select([c for c in lf_enr.columns  if c not in DROP_NOISE])

# 欄位集合
orig_cols = set(lf_orig.columns)
enr_cols  = set(lf_enr.columns)

# 2) 找出 enriched 缺的欄位，從 orig 依 Id 左連接補回
missing_in_enr = sorted(orig_cols - enr_cols)
if missing_in_enr:
    lf_enr = lf_enr.join(
        lf_orig.select(["Id"] + missing_in_enr),
        on="Id",
        how="left",
    )

# 3) 共同欄位 dtype 對齊到 orig 的 dtype
orig_schema   = lf_orig.schema  # dict: {col: dtype}
common_cols   = sorted(orig_cols & set(lf_enr.columns))
cast_exprs    = [pl.col(c).cast(orig_schema[c], strict=False) for c in common_cols]

lf_enr_fixed = lf_enr.with_columns(cast_exprs)

# 寫檔（串流，不 materialize）
os.makedirs(os.path.dirname(OUT_ENRICHED_FIX), exist_ok=True)
lf_enr_fixed.sink_parquet(OUT_ENRICHED_FIX, compression="zstd")
print("✅ saved to:", OUT_ENRICHED_FIX)


C:\Users\wani\AppData\Local\Temp\ipykernel_21804\813536562.py:17: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  lf_orig = lf_orig.select([c for c in lf_orig.columns if c not in DROP_NOISE])
C:\Users\wani\AppData\Local\Temp\ipykernel_21804\813536562.py:18: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  lf_enr  = lf_enr.select([c for c in lf_enr.columns  if c not in DROP_NOISE])
C:\Users\wani\AppData\Local\Temp\ipykernel_21804\813536562.py:21: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the 

✅ saved to: data/data_from_json/train/train_all_columns_enriched_fixed.parquet


In [16]:
train_filled['legs1_segments0_baggageAllowance_weightMeasurementType'].unique()

legs1_segments0_baggageAllowance_weightMeasurementType
f64
null
0.0
1.0


In [3]:
train_filled.select(sig_cols)

totalPrice,taxes,legs0_departureAt,legs0_arrivalAt,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_departureFrom_airport_iata,legs0_segments0_arrivalTo_airport_iata,legs1_departureAt,legs1_arrivalAt,legs1_segments0_flightNumber,legs1_segments0_marketingCarrier_code,legs1_segments0_departureFrom_airport_iata,legs1_segments0_arrivalTo_airport_iata,legs0_segments1_flightNumber,legs1_segments1_flightNumber,legs1_segments0_seatsAvailable,legs0_segments0_seatsAvailable,legs1_segments1_seatsAvailable,legs0_segments1_seatsAvailable,legs1_segments0_baggageAllowance_quantity,corporateTariffCode,legs0_segments0_baggageAllowance_quantity,legs0_segments1_marketingCarrier_code,legs0_segments2_flightNumber,miniRules0_monetaryAmount,miniRules1_monetaryAmount,legs1_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_quantity,legs1_segments2_baggageAllowance_quantity,legs0_segments2_baggageAllowance_quantity,legs1_segments2_flightNumber,legs1_segments0_baggageAllowance_weightMeasurementType,miniRules0_statusInfos,legs0_segments2_seatsAvailable,legs0_segments1_duration
f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,i64,f64,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,str
16884.0,370.0,"""2024-06-15T15:40:00""","""2024-06-15T16:20:00""","""216""","""KV""","""TLK""","""KJA""","""2024-07-09T09:45:00""","""2024-07-09T14:20:00""","""215""","""KV""","""KJA""","""TLK""",null,null,9.0,9.0,null,null,1.0,null,1.0,null,null,null,null,null,null,null,null,null,0.0,null,null,null
51125.0,2240.0,"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""5358""","""S7""","""TLK""","""OVB""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""5338""","""S7""","""KJA""","""OVB""","""5311""","""5357""",9.0,4.0,9.0,4.0,1.0,123,1.0,"""S7""",null,2300.0,3500.0,1.0,1.0,null,null,null,0.0,1.0,null,"""01:20:00"""
53695.0,2240.0,"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""5358""","""S7""","""TLK""","""OVB""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""5338""","""S7""","""KJA""","""OVB""","""5311""","""5357""",9.0,4.0,9.0,4.0,1.0,null,1.0,"""S7""",null,2300.0,3500.0,1.0,1.0,null,null,null,0.0,1.0,null,"""01:20:00"""
81880.0,2240.0,"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""5358""","""S7""","""TLK""","""OVB""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""5338""","""S7""","""KJA""","""OVB""","""5311""","""5357""",9.0,4.0,9.0,4.0,1.0,123,1.0,"""S7""",null,0.0,0.0,1.0,1.0,null,null,null,0.0,1.0,null,"""01:20:00"""
86070.0,2240.0,"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""5358""","""S7""","""TLK""","""OVB""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""5338""","""S7""","""KJA""","""OVB""","""5311""","""5357""",9.0,4.0,9.0,4.0,1.0,null,1.0,"""S7""",null,0.0,0.0,1.0,1.0,null,null,null,0.0,1.0,null,"""01:20:00"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
30730.0,5560.0,"""2024-11-05T20:50:00""","""2024-11-06T09:45:00""","""247""","""UT""","""VKO""","""SGC""","""2024-11-09T19:25:00""","""2024-11-10T08:10:00""","""1105""","""UT""","""SVX""","""UFA""","""111""","""364""",8.0,3.0,1.0,5.0,0.0,44,0.0,"""UT""",null,24000.0,0.0,0.0,0.0,null,null,null,0.0,1.0,null,"""02:30:00"""
27660.0,5560.0,"""2024-11-05T00:20:00""","""2024-11-05T20:00:00""","""363""","""UT""","""VKO""","""UFA""","""2024-11-09T21:10:00""","""2024-11-10T08:35:00""","""112""","""UT""","""SVX""","""SGC""","""106""","""248""",4.0,3.0,3.0,9.0,0.0,44,0.0,"""UT""",null,24000.0,0.0,0.0,0.0,null,null,null,0.0,1.0,null,"""01:30:00"""
24460.0,5460.0,"""2024-11-05T00:20:00""","""2024-11-05T20:00:00""","""363""","""UT""","""VKO""","""UFA""","""2024-11-09T19:25:00""","""2024-11-10T08:10:00""","""1105""","""UT""","""SVX""","""UFA""","""106""","""364""",8.0,3.0,1.0,9.0,0.0,44,0.0,"""UT""",null,24000.0,0.0,0.0,0.0,null,null,null,0.0,1.0,null,"""01:30:00"""


In [9]:
df

totalPrice,taxes,legs0_departureAt,legs0_arrivalAt,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_departureFrom_airport_iata,legs0_segments0_arrivalTo_airport_iata,legs1_departureAt,legs1_arrivalAt,legs1_segments0_flightNumber,legs1_segments0_marketingCarrier_code,legs1_segments0_departureFrom_airport_iata,legs1_segments0_arrivalTo_airport_iata,legs0_segments1_flightNumber,legs1_segments1_flightNumber,legs1_segments0_seatsAvailable,legs0_segments0_seatsAvailable,legs1_segments1_seatsAvailable,legs0_segments1_seatsAvailable,legs1_segments0_baggageAllowance_quantity,corporateTariffCode,legs0_segments0_baggageAllowance_quantity,legs0_segments1_marketingCarrier_code,legs0_segments2_flightNumber,miniRules0_monetaryAmount,miniRules1_monetaryAmount,legs1_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_quantity,legs1_segments2_baggageAllowance_quantity,legs0_segments2_baggageAllowance_quantity,legs1_segments2_flightNumber,legs1_segments0_baggageAllowance_weightMeasurementType,miniRules0_statusInfos,legs0_segments2_seatsAvailable,legs0_segments1_duration
f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,str,i64,bool,i64,str
3717.0,507.0,"""2024-09-23T23:55:00""","""2024-09-24T01:30:00""","""6708""","""SU""","""LED""","""SVO""",null,null,null,null,null,null,null,null,null,1,null,null,null,57,0,null,null,2800,0,null,null,null,null,null,null,true,null,null
4317.0,507.0,"""2024-09-23T23:55:00""","""2024-09-24T01:30:00""","""6708""","""SU""","""LED""","""SVO""",null,null,null,null,null,null,null,null,null,1,null,null,null,57,0,null,null,2800,0,null,null,null,null,null,null,true,null,null
5542.0,507.0,"""2024-09-23T23:55:00""","""2024-09-24T01:30:00""","""6708""","""SU""","""LED""","""SVO""",null,null,null,null,null,null,null,null,null,1,null,null,null,57,1,null,null,2800,2800,null,null,null,null,null,null,true,null,null
9632.0,507.0,"""2024-09-23T23:55:00""","""2024-09-24T01:30:00""","""6708""","""SU""","""LED""","""SVO""",null,null,null,null,null,null,null,null,null,1,null,null,null,57,2,null,null,0,0,null,null,null,null,null,null,true,null,null
19407.0,507.0,"""2024-09-23T23:55:00""","""2024-09-24T01:30:00""","""6708""","""SU""","""LED""","""SVO""",null,null,null,null,null,null,null,null,null,4,null,null,null,57,1,null,null,2800,0,null,null,null,null,null,null,true,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
63414.0,2824.0,"""2024-10-25T19:25:00""","""2024-10-26T13:05:00""","""5346""","""S7""","""OMS""","""OVB""","""2024-11-10T13:50:00""","""2024-11-11T10:25:00""","""5004""","""S7""","""LED""","""OVB""","""5003""","""5343""",9,9,9,1,1,null,1,"""S7""",null,2300,3500,1,1,null,null,null,0,true,null,"""04:30:00"""
115549.0,2824.0,"""2024-10-25T19:25:00""","""2024-10-26T13:05:00""","""5346""","""S7""","""OMS""","""OVB""","""2024-11-10T13:50:00""","""2024-11-11T10:25:00""","""5004""","""S7""","""LED""","""OVB""","""5003""","""5343""",9,9,9,9,1,null,1,"""S7""",null,0,0,1,1,null,null,null,0,true,null,"""04:30:00"""
51154.0,2824.0,"""2024-10-25T19:25:00""","""2024-10-26T13:05:00""","""5346""","""S7""","""OMS""","""OVB""","""2024-11-10T16:40:00""","""2024-11-11T18:35:00""","""5006""","""S7""","""LED""","""OVB""","""5003""","""5345""",7,9,9,1,0,null,0,"""S7""",null,4000,0,0,0,null,null,null,0,true,null,"""04:30:00"""


In [2]:
side = pl.read_parquet(SIDE_MAP_PATH)
side

_idx,Id
u32,i64
61,null
79,null
137,null
187,null
215,null
…,…
17956780,null
17956794,null
17956800,null


In [4]:
import polars as pl

SIDE_MAP_PATH = "data/_id_side_map.parquet"

side = pl.read_parquet(SIDE_MAP_PATH)  # 欄位：_idx, Id

n_rows   = side.height
n_null   = side["Id"].null_count()
n_nn     = n_rows - n_null
n_unique = side.filter(pl.col("Id").is_not_null()).select(pl.col("Id").n_unique()).item()
dupes    = n_nn - n_unique

# 各 Id 出現次數（只看非空）
counts = (
    side.filter(pl.col("Id").is_not_null())
        .group_by("Id")
        .len()
        .sort("len", descending=True)
)

max_count = counts.select(pl.col("len").max()).item() if counts.height else 0

print(f"rows(total): {n_rows:,}")
print(f"Id null    : {n_null:,}")
print(f"Id non-null: {n_nn:,}")
print(f"Id unique  : {n_unique:,}")
print(f"Id duplicates (non-null - unique): {dupes:,}")
print(f"max occurrences of a single Id   : {max_count:,}")

# 列出前 10 個重複 Id 的樣子
top_dupes = counts.filter(pl.col("len") > 1).head(10)
if top_dupes.height:
    sample = (
        side.join(top_dupes.select("Id"), on="Id", how="inner")
            .group_by("Id")
            .agg([
                pl.len().alias("n"),
                pl.col("_idx").head(5).alias("sample_idx")  # 每個 Id 範例的前 5 個 _idx
            ])
            .sort("n", descending=True)
    )
    print("\nTop duplicate Ids (n, sample_idx):")
    print(sample)

# 簡答你的問題：
if max_count == n_rows and n_rows > 0:
    print("\n⚠️ 注意：max_count == len(side)。代表所有列都被指到同一個 Id，這幾乎可以確定是簽名撞在一起了。")
elif max_count > 1:
    print("\nℹ️ 有些 Id 對到多於一列（簽名不夠唯一）；理想情況是 max_count == 1。")
else:
    print("\n✅ 每個 Id 只對到一列（max_count == 1），這是理想狀態。")


rows(total): 18,151,423
Id null    : 18,151,423
Id non-null: 0
Id unique  : 0
Id duplicates (non-null - unique): 0
max occurrences of a single Id   : 0

✅ 每個 Id 只對到一列（max_count == 1），這是理想狀態。


In [2]:
df

BestPrice,BestPriceCorporateTariff,BestPriceDirect,BestPriceTravelPolicy,Convenience,MinTime,bySelf,category,companyID,corporateTariffCode,currencyCode,frequentFlyer,grade,hasAssistant,industryEN,isGlobal,isSegDiscountAsExactValue,isSegDiscountVariant,isVip,legs0_arrivalAt,legs0_arrivalTo_airport_city_code,legs0_arrivalTo_airport_city_country_codeA2,legs0_arrivalTo_airport_city_country_codeA3,legs0_arrivalTo_airport_city_iata,legs0_arrivalTo_airport_code,legs0_arrivalTo_airport_iata,legs0_arrivalTo_airport_icao,legs0_arrivalTo_terminal,legs0_departureAt,legs0_departureFrom_airport_city_code,legs0_departureFrom_airport_city_country_codeA2,legs0_departureFrom_airport_city_country_codeA3,legs0_departureFrom_airport_city_iata,legs0_departureFrom_airport_code,legs0_departureFrom_airport_iata,legs0_departureFrom_airport_icao,legs0_departureFrom_terminal,…,legs1_segments3_transfer,legs1_segments_count,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,nationality,option_legs_count,ordinal,pointOfSale,position,pricingInfo_count,pricingInfo_isAccessTP,pricingInfo_passengerCount,pricingInfo_passengerType,pricingInfo_price,pricingInfo_selfPaid,pricingInfo_taxes,pricing_present,profileId,ranker_id,requestDate,requestDepartureDate,requestReturnDate,request_time,root_$id,searchRoute,searchType,sex,supplierId,taxes,timeLimit,totalPrice,validatingCarrier,yearOfBirth
bool,bool,bool,bool,bool,bool,bool,i64,i64,i64,str,str,null,bool,null,bool,bool,bool,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,null,i64,i64,i64,bool,i64,i64,bool,i64,i64,i64,null,null,i64,bool,i64,i64,f64,bool,f64,bool,i64,str,str,str,str,str,str,str,i64,bool,str,f64,str,f64,str,i64
true,true,true,true,false,false,true,15,40253,57,"""RUB""","""SU""",null,false,null,false,false,false,true,"""2024-09-24T01:30:00""","""MOW""","""RU""","""RUS""","""MOW""","""SVO""","""SVO""","""UUEE""","""B""","""2024-09-23T23:55:00""","""LED""","""RU""","""RUS""","""LED""","""LED""","""LED""","""ULLI""","""1""",…,null,0,2800,null,true,0,null,false,36,1,0,null,null,1,true,1,0,3717.0,false,507.0,true,2384236,"""99d7fb3f49c642f0b4d45d2156271d…","""2024-08-27T11:43:41""","""2024-09-23T00:00:00""",null,"""2024-08-27T08:43:41.447890""","""1""","""LEDMOW""",1,true,"""e294cac4-ea1b-4431-bc5c-464118…",507.0,"""2024-08-29T11:43:00+03:00""",3717.0,"""SU""",1980
false,false,false,false,false,false,true,0,40253,57,"""RUB""","""SU""",null,false,null,false,false,false,true,"""2024-09-24T01:30:00""","""MOW""","""RU""","""RUS""","""MOW""","""SVO""","""SVO""","""UUEE""","""B""","""2024-09-23T23:55:00""","""LED""","""RU""","""RUS""","""LED""","""LED""","""LED""","""ULLI""","""1""",…,null,0,2800,null,true,0,null,false,36,1,1,null,null,1,true,1,0,4317.0,false,507.0,true,2384236,"""99d7fb3f49c642f0b4d45d2156271d…","""2024-08-27T11:43:41""","""2024-09-23T00:00:00""",null,"""2024-08-27T08:43:41.447890""","""1""","""LEDMOW""",1,true,"""e294cac4-ea1b-4431-bc5c-464118…",507.0,"""2024-08-29T11:43:00+03:00""",4317.0,"""SU""",1980
false,false,false,false,false,false,true,0,40253,57,"""RUB""","""SU""",null,false,null,false,false,false,true,"""2024-09-24T01:30:00""","""MOW""","""RU""","""RUS""","""MOW""","""SVO""","""SVO""","""UUEE""","""B""","""2024-09-23T23:55:00""","""LED""","""RU""","""RUS""","""LED""","""LED""","""LED""","""ULLI""","""1""",…,null,0,2800,null,true,2800,null,true,36,1,2,null,null,1,true,1,0,5542.0,false,507.0,true,2384236,"""99d7fb3f49c642f0b4d45d2156271d…","""2024-08-27T11:43:41""","""2024-09-23T00:00:00""",null,"""2024-08-27T08:43:41.447890""","""1""","""LEDMOW""",1,true,"""e294cac4-ea1b-4431-bc5c-464118…",507.0,"""2024-08-29T11:43:00+03:00""",5542.0,"""SU""",1980
false,false,false,false,false,false,true,0,40253,57,"""RUB""","""SU""",null,false,null,false,false,false,true,"""2024-09-24T01:30:00""","""MOW""","""RU""","""RUS""","""MOW""","""SVO""","""SVO""","""UUEE""","""B""","""2024-09

In [ ]:
import polars as pl

TRAIN_FILLED_PATH = "data/train.parquet"
OUT_PATH          = "data/train_all_columns_with_id.parquet"
SIDE_MAP_PATH     = "data/_id_side_map.parquet"

train = pl.read_parquet(TRAIN_FILLED_PATH, columns=["ranker_id","Id","totalPrice","currencyCode"])
side  = pl.read_parquet(SIDE_MAP_PATH)  # (_idx, Id)
df_id = pl.read_parquet(OUT_PATH, columns=["ranker_id","Id","totalPrice","currencyCode"])

# 1) 側檔內是否有任何 null Id？（100% 的話應該是 0）
print("null Id count in side map:", side["Id"].null_count())

# 2) 每個 _sig 是否只對到一個 Id？（如果你有保留 sig_map 也可檢查 n_unique==1）
#    這裡退而求其次：檢查 (ranker_id, Id) 在 df 端是否唯一（通常應該唯一）
dupes = (
    df_id.group_by(["ranker_id","Id"])
         .len()
         .filter(pl.col("len") > 1)
)
print("duplicate (ranker_id, Id) rows in df:", dupes.height)

# 3) df 與 train 的 (ranker_id, Id) 交集是否一致？（反連接應該為 0）
anti_df  = df_id.join(train.select(["ranker_id","Id"]), on=["ranker_id","Id"], how="anti")
anti_tr  = train.select(["ranker_id","Id"]).join(df_id, on=["ranker_id","Id"], how="anti")
print("in df but not in train:", anti_df.height)
print("in train but not in df:", anti_tr.height)

# 4) 非簽名欄位的抽查（挑幾個沒拿來做簽名的欄位來比對；這裡示範 taxes）
#    如果 OUT_PATH 裡有 taxes，就抽 1000 列對齊 train 的 taxes 是否也一致（可容忍少量差）
cols_to_probe = ["taxes"]  # 你可以換成其他沒拿去做簽名的欄位
probe_cols = [c for c in cols_to_probe if c in df_id.columns and c in train.columns]
if probe_cols:
    merged = (
        df_id.select(["ranker_id","Id"] + probe_cols)
             .join(train.select(["ranker_id","Id"] + probe_cols), on=["ranker_id","Id"], how="inner", suffix="_train")
             .sample(n=1000, shuffle=True)
    )
    for c in probe_cols:
        mism = merged.filter(pl.col(c) != pl.col(f"{c}_train"))
        print(f"probe mismatch for {c}:", mism.height)
else:
    print("no probe columns found to compare")


In [92]:
df

root_$id,ranker_id,request_time,searchType,profileId,sex,bySelf,yearOfBirth,nationality,companyID,isVip,hasAssistant,isGlobal,frequentFlyer,position,pointOfSale,industryEN,grade,requestDate,searchRoute,requestDepartureDate,requestReturnDate,validatingCarrier,category,option_legs_count,legs0_duration,legs0_departureAt,legs0_arrivalAt,legs0_departureFrom_airport_code,legs0_departureFrom_airport_iata,legs0_departureFrom_airport_icao,legs0_departureFrom_airport_city_code,legs0_departureFrom_airport_city_iata,legs0_departureFrom_airport_city_country_codeA2,legs0_departureFrom_airport_city_country_codeA3,legs0_departureFrom_terminal,legs0_arrivalTo_airport_code,…,legs1_segments3_arrivalTo_airport_city_country_codeA2,legs1_segments3_arrivalTo_airport_city_country_codeA3,legs1_segments3_arrivalTo_terminal,pricing_present,totalPrice,taxes,timeLimit,supplierId,ordinal,currencyCode,corporateTariffCode,isSegDiscountVariant,isSegDiscountAsExactValue,MinTime,BestPriceCorporateTariff,BestPriceTravelPolicy,BestPrice,Convenience,BestPriceDirect,monetaryAmount,statusInfos,miniRules_currencyCode,pricingInfo_count,legs0_segments0_ticketDesignator,legs0_segments0_class,legs0_segments0_cabinClass,legs0_segments0_seatsAvailable,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_type,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments1_ticketDesignator,legs0_segments1_class,legs0_segments1_cabinClass,legs0_segments1_seatsAvailable,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_type,legs0_segments1_baggageAllowance_weightMeasurementType
str,str,str,i64,i64,bool,bool,i64,i64,i64,bool,bool,bool,str,null,null,null,null,str,str,str,null,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,…,null,null,null,bool,i64,i64,str,str,i64,str,null,bool,bool,bool,bool,bool,bool,bool,bool,i64,bool,str,i64,null,str,i64,i64,i64,i64,i64,null,str,i64,i64,i64,i64,i64
"""1""","""000a8ad3b98d4b41bcddcb944357e3…","""2024-10-23T09:43:14.488319""",1,694838,true,true,1988,36,62063,false,true,false,"""SU/EK""",null,null,null,null,"""2024-10-23T09:43:14""","""MOWIST""","""2024-10-27T00:00:00""",null,"""U6""",33,1,"""10:10:00""","""2024-10-27T05:25:00""","""2024-10-27T15:35:00""","""ZIA""","""ZIA""","""UUBW""","""MOW""","""MOW""","""RU""","""RUS""",null,"""IST""",…,null,null,null,true,77824,4984,"""2024-10-23T10:13:00+00:00""","""0223d886-9910-4273-934f-68f2d6…",0,"""RUB""",null,false,false,false,false,false,true,false,false,0,true,null,1,null,"""C""",2,2,2,2,0,null,"""C""",2,2,2,2,0
"""1""","""000a8ad3b98d4b41bcddcb944357e3…","""2024-10-23T09:43:14.488319""",1,694838,true,true,1988,36,62063,false,true,false,"""SU/EK""",null,null,null,null,"""2024-10-23T09:43:14""","""MOWIST""","""2024-10-27T00:00:00""",null,"""SU""",33,1,"""05:30:00""","""2024-10-27T09:40:00""","""2024-10-27T15:10:00""","""SVO""","""SVO""","""UUEE""","""MOW""","""MOW""","""RU""","""RUS""","""C""","""IST""",…,null,null,null,true,86919,1019,"""2024-10-23T10:14:00+00:00""","""7843589e-2da5-43fc-8780-87b174…",1,"""RUB""",null,false,false,false,false,false,false,true,true,0,false,null,1,null,"""C""",2,2,1,2,0,null,null,null,null,null,null,null
"""1""","""000a8ad3b98d4b41bcddcb944357e3…","""2024-10-23T09:43:14.488319""",1,694838,true,true,1988,36,62063,false,true,false,"""SU/EK""",null,null,null,null,"""2024-10-23T09:43:14""","""MOWIST""","""2024-10-27T00:00:00""",null,"""SU""",33,1,"""05:30:00""","""2024-10-27T09:40:00""","""2024-10-27T15:10:00""","""SVO""","""SVO""","""UUEE""","""MOW""","""MOW""","""RU""","""RUS""","""C""","""IST""",…,null,null,null,true,99774,1019,"""2024-10-23T10:14:00+00:00""","""7843589e-2da5-43fc-8780-87b174…",2,"""RUB""",null,false,false,false,false,false,false,false,false,4600,true,"""RUB""",1,null,"""C""",2,2,2,2,0,null,null,null,null,null,null,null
"""1""","""000a8ad3b98d4b41bcddcb944357e3…","""2024-10-23T09:43:14.488319""",1,694838,true,true,1988,36,62063,false,true,false,"""SU/EK""",null,null,null,null,"""2024-10

In [93]:
df.write_csv('data/all_columns_sample.csv')


In [123]:
df['miniRules0_monetaryAmount', 'pricingInfo_isAccessTP']

miniRules0_monetaryAmount,pricingInfo_isAccessTP
i64,null
0,null
4600,null
4600,null
0,null
6270,null
…,…
0,null
0,null
0,null


In [48]:
df['ranker_id','leg0_segments1_baggageAllowance_type', 'leg0_segments1_baggageAllowance_quantity','leg0_segments1_baggageAllowance_weightMeasurementType']

ranker_id,leg0_segments1_baggageAllowance_type,leg0_segments1_baggageAllowance_quantity,leg0_segments1_baggageAllowance_weightMeasurementType
str,i64,i64,i64
"""000a8ad3b98d4b41bcddcb944357e3…",2,2,0
"""000a8ad3b98d4b41bcddcb944357e3…",null,null,null
"""000a8ad3b98d4b41bcddcb944357e3…",null,null,null
"""000a8ad3b98d4b41bcddcb944357e3…",null,null,null
"""000a8ad3b98d4b41bcddcb944357e3…",null,null,null
…,…,…,…
"""000a8ad3b98d4b41bcddcb944357e3…",1,40,1
"""000a8ad3b98d4b41bcddcb944357e3…",1,40,1
"""000a8ad3b98d4b41bcddcb944357e3…",1,40,1


In [100]:
train_filled = pl.read_parquet("data/train.parquet")


In [ ]:
import polars as pl

def compare_columns(df_left: pl.DataFrame, df_right: pl.DataFrame,
                    left_name: str = "df", right_name: str = "train_filled"):
    left_cols  = set(df_left.columns)
    right_cols = set(df_right.columns)

    common      = sorted(left_cols & right_cols)
    only_left   = sorted(left_cols - right_cols)
    only_right  = sorted(right_cols - left_cols)

    # 報告摘要
    print(f"🧮 {left_name} columns:  {len(left_cols)}")
    print(f"🧮 {right_name} columns: {len(right_cols)}")
    print(f"✅ common:               {len(common)}")
    if left_cols:
        print(f"   coverage on {left_name}:  {len(common)/len(left_cols):.1%}")
    if right_cols:
        print(f"   coverage on {right_name}: {len(common)/len(right_cols):.1%}")

    # 也回傳成 DataFrame，方便檢視或存檔
    common_df     = pl.DataFrame({"common_columns": common})
    only_left_df  = pl.DataFrame({f"only_in_{left_name}": only_left})
    only_right_df = pl.DataFrame({f"only_in_{right_name}": only_right})

    return common_df, only_left_df, only_right_df

# 假設你已經有 df 以及讀好的 train_filled
train_filled = pl.read_parquet("data/train.parquet")
df = pl.read_parquet("data/train_all_columns.parquet")

common_df, only_in_df, only_in_train = compare_columns(df, train_filled)

# 看前幾列
print("\n— common columns (head) —")
print(common_df.head(20))

print("\n— only in df (head) —")
print(only_in_df.head(20))

print("\n— only in train_filled (head) —")
print(only_in_train.head(20))

# 若想輸出成 CSV 方便比對
# common_df.write_csv("common_columns.csv")
# only_in_df.write_csv("only_in_df.csv")
# only_in_train.write_csv("only_in_train_filled.csv")


🧮 df columns:  358
🧮 train_filled columns: 127
✅ common:               123
   coverage on df:  34.4%
   coverage on train_filled: 96.9%

— common columns (head) —
shape: (20, 1)
┌─────────────────────────────────┐
│ common_columns                  │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ bySelf                          │
│ companyID                       │
│ corporateTariffCode             │
│ frequentFlyer                   │
│ isVip                           │
│ …                               │
│ legs0_segments0_duration        │
│ legs0_segments0_flightNumber    │
│ legs0_segments0_marketingCarri… │
│ legs0_segments0_operatingCarri… │
│ legs0_segments0_seatsAvailable  │
└─────────────────────────────────┘

— only in df (head) —
shape: (20, 1)
┌─────────────────────────────────┐
│ only_in_df                      │
│ ---                             │
│ str                             │
╞═══════════════════════════

In [113]:
only_in_train

only_in_train_filled
str
"""Id"""
"""__index_level_0__"""
"""isAccess3D"""
"""legs0_segments2_baggageAllowan…"
"""legs0_segments2_baggageAllowan…"
…
"""legs1_segments3_cabinClass"""
"""legs1_segments3_seatsAvailable"""
"""pricingInfo_isAccessTP"""


In [125]:
import polars as pl

rid = "000a8ad3b98d4b41bcddcb944357e3f0"

# 你想看的主要欄位（官方命名）
target_cols = [
    "miniRules0_monetaryAmount",   # 你現在問的這個

]

# 找到第一個存在的欄位名稱
existing = next((c for c in target_cols if c in train_filled.columns), None)

if existing is None:
    # 沒有精準命名時，列出所有與 segments1 + bag 有關的欄位，方便你挑
    bag_like = [c for c in train_filled.columns
                if ("segments1" in c) and ("bag" in c.lower() or "baggage" in c.lower())]
    print("⚠️ 找不到精確欄位，以下是相近欄位可用：")
    for c in bag_like:
        print(" -", c)
else:
    # 1) 直接取值（可能有多列，因為一個 ranker 可能對應多個 pricing）
    out = (
        train_filled
        .filter(pl.col("ranker_id") == rid)
        .select(existing)
    )
    display(out.to_pandas().head(20))  # 看前 20 筆

    # 2) 只看唯一值（快速確認）
    uniq_vals = (
        train_filled
        .filter(pl.col("ranker_id") == rid)
        .select(pl.col(existing).unique().alias(f"unique_{existing}"))
    )
    print("Unique values:")
    display(uniq_vals.to_pandas())

    # 3) 如果想同時看這筆 ranker 的所有與 bag 相關的段資訊
    bag_cols = [c for c in train_filled.columns
                if c.startswith("leg") and "segments" in c and ("bag" in c.lower() or "baggage" in c.lower())]
    if bag_cols:
        bag_view = (
            train_filled
            .filter(pl.col("ranker_id") == rid)
            .select(["ranker_id"] + bag_cols)
        )
        print(f"\nAll bag-related segment fields for ranker_id={rid}:")
        display(bag_view.head(5).to_pandas())


,miniRules0_monetaryAmount
0,0.0
1,4600.0
2,4600.0
3,0.0
4,6270.0
5,0.0
6,4600.0
7,4600.0
8,0.0
9,4600.0


Unique values:


,unique_miniRules0_monetaryAmount
0,0.0
1,4180.0
2,4600.0
3,5515.0
4,6270.0
5,7965.0
6,8685.0
7,9405.0
8,12020.0
9,15675.0



All bag-related segment fields for ranker_id=000a8ad3b98d4b41bcddcb944357e3f0:


,ranker_id,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments2_baggageAllowance_quantity,legs0_segments2_baggageAllowance_weightMeasurementType,legs0_segments3_baggageAllowance_quantity,legs0_segments3_baggageAllowance_weightMeasurementType,legs1_segments0_baggageAllowance_quantity,legs1_segments0_baggageAllowance_weightMeasurementType,legs1_segments1_baggageAllowance_quantity,legs1_segments1_baggageAllowance_weightMeasurementType,legs1_segments2_baggageAllowance_quantity,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType
0,000a8ad3b98d4b41bcddcb944357e3f0,2.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000a8ad3b98d4b41bcddcb944357e3f0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000a8ad3b98d4b41bcddcb944357e3f0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000a8ad3b98d4b41bcddcb944357e3f0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000a8ad3b98d4b41bcddcb944357e3f0,30.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
